# MASTER NOTEBOOK - EXPLORATORY ANALYSIS & BASELINE METRICS
## Final Project - Word Sense Disambiguation
### Chris Caudill, Stephanie Fan, Kent Owen

### GET RELEVANT DATA FROM "KEY" FILE

In [ ]:
"""ADDED TRAIN/TEST 8-12-17"""

#GET KEYS - UNCOMMENT TO GET RELEVANT DATASET

#SEMCOR ONLY
with open ("WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.gold.key_orig.txt") as f :
    semcor_keys = f.readlines()

#SEMCOR+OMSTI - COMMENT OUT IF DON'T NEED (TAKES A LONG TIME)
# with open ("WSD_Evaluation_Framework/Training_Corpora/SemCor+OMSTI/semcor+omsti.gold.key.txt") as f :
#     semcor_omsti_keys = f.readlines()

#EVAL SETS
with open ("WSD_Evaluation_Framework/Evaluation_Datasets/semeval2007/semeval2007.gold.key.txt") as f :
    semeval2007_keys = f.readlines()
with open ("WSD_Evaluation_Framework/Evaluation_Datasets/semeval2013/semeval2013.gold.key.txt") as f :
    semeval2013_keys = f.readlines()
with open ("WSD_Evaluation_Framework/Evaluation_Datasets/semeval2015/semeval2015.gold.key.txt") as f :
    semeval2015_keys = f.readlines()
with open ("WSD_Evaluation_Framework/Evaluation_Datasets/senseval2/senseval2.gold.key.txt") as f :
    senseval2_keys = f.readlines()
with open ("WSD_Evaluation_Framework/Evaluation_Datasets/senseval3/senseval3.gold.key.txt") as f :
    senseval3_keys = f.readlines()
    
#SEMCOR TRAIN
## COMMENT OUT IF NOT ALREADY CREATED
with open ("WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.gold.key.train.txt") as f :
    semcortrain_keys = f.readlines()

with open ("WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.gold.key.test.txt") as f :
    semcortest_keys = f.readlines()


print "KEYS EXAMPLES"
print semcor_keys[0:10]
print semeval2007_keys[0:10]

### SPLIT KEYS DATA INTO TRAIN/TEST

In [ ]:
"""KO MODIFIED 8/12/17 - CREATE TRAIN/TEST KEYS AND GET RID OF 2nd WORD SENSE ASSIGNMENT"""

keys_input = semcor_keys
output_folder = "WSD_Evaluation_Framework/Training_Corpora/SemCor/"
train_name = "semcor.gold.key.train.txt"
test_name = "semcor.gold.key.test.txt"
train_path = output_folder+train_name
test_path = output_folder+test_name

with open(train_path,"w") as f :
    with open (test_path, "w") as f2 :
        for line in keys_input :
            line = line.strip('\n')
            ws_count = line.count(' ')    
            #delete 2nd+ word sense assignment(s)
            if ws_count > 1 :
                index_1 = line.find(' ')
                index_2 = (line[index_1+1:].find(' ')+1)+index_1
                line = line[:index_2]            
            instance = line.split(".",2)
            article = instance[0]
            sentence = instance[1]
            tag = instance[2]
            article_num = int(article[1:])
            if article_num %100 <> 0 :
                f.write(line+'\n')
            else :
                f2.write(line+'\n')

with open (train_path, "r") as f :
    train_keys = f.readlines()
    
    
with open (test_path, "r") as f2 :
    test_keys = f2.readlines()
    
print train_keys[0:10]
print len(train_keys)
print len(test_keys)

### CLEAN UP EVAL SET KEY FILES

In [ ]:
## 8/22 CC: Added function to take less space in notebook

def cleanupKeyFiles(data):

    if data=='SemCor':
        keys_input = semcor_keys
        output_folder = "WSD_Evaluation_Framework/Training_Corpora/SemCor/"
        orig_name = "semcor.gold.key_orig.txt"
        clean_name = "semcor.gold.key.txt"

    elif data=='SemEval2007':
        keys_input = semeval2007_keys
        output_folder = "WSD_Evaluation_Framework/Evaluation_Datasets/semeval2007/"
        orig_name = "semeval2007.gold.key_orig.txt"
        clean_name = "semeval2007.gold.key.txt"

    elif data=='SemEval2013':
        keys_input = semeval2013_keys
        output_folder = "WSD_Evaluation_Framework/Evaluation_Datasets/semeval2013/"
        orig_name = "semeval2013.gold.key_orig.txt"
        clean_name = "semeval2013.gold.key.txt"


    elif data=='SemEval2015':
        keys_input = semeval2015_keys
        output_folder = "WSD_Evaluation_Framework/Evaluation_Datasets/semeval2015/"
        orig_name = "semeval2015.gold.key_orig.txt"
        clean_name = "semeval2015.gold.key.txt"


    elif data=='SensEval2':
        keys_input = senseval2_keys
        output_folder = "WSD_Evaluation_Framework/Evaluation_Datasets/senseval2/"
        orig_name = "senseval2.gold.key_orig.txt"
        clean_name = "senseval2.gold.key.txt"


    elif data=='SensEval3':
        keys_input = senseval3_keys
        output_folder = "WSD_Evaluation_Framework/Evaluation_Datasets/senseval3/"
        orig_name = "senseval3.gold.key_orig.txt"
        clean_name = "senseval3.gold.key.txt"


    orig_path = output_folder+orig_name
    clean_path = output_folder+clean_name

    with open(orig_path,"r") as f_orig :
        with open (clean_path, "w") as f_clean :
            for line in keys_input :
                line = line.strip('\n')
                ws_count = line.count(' ')    
                #delete 2nd+ word sense assignment(s)
                if ws_count > 1 :
                    index_1 = line.find(' ')
                    index_2 = (line[index_1+1:].find(' ')+1)+index_1
                    line = line[:index_2]            
                instance = line.split(".",2)
                article = instance[0]
                sentence = instance[1]
                tag = instance[2]
                article_num = int(article[1:])
                f_clean.write(line+'\n')
    return "Done"


cleanupKeyFiles('SemCor')
cleanupKeyFiles('SemEval2007')
cleanupKeyFiles('SemEval2013')
cleanupKeyFiles('SemEval2015')
cleanupKeyFiles('SensEval2')
cleanupKeyFiles('SensEval3')

### BUILD ONE-HOT ENCODING "VOCAB" OF WORD SENSE ASSIGNMENTS BASED ON TRAINING DATA

In [ ]:
"""MODIFIED 8_15_17"""

# Helper libraries
from shared_lib import utils, vocabulary, ws_encoder

import itertools
reload(utils)
reload(ws_encoder)

def get_senses(keys) :
    ## MUST USE "CLEAN" VERSION OF KEYS WITH JUST A SINGLE WORD SENSE
    word_senses = [line.split(' ')[1].rstrip() for line in keys]
    return word_senses
    
#get just the word sense assignments (like 'be%2:42:03::\n')
train_word_senses = get_senses(semcortrain_keys)
semeval2007_test_ws = get_senses(semeval2007_keys)
semeval2013_test_ws = get_senses(semeval2013_keys)
semeval2015_test_ws = get_senses(semeval2015_keys)
senseval2_test_ws = get_senses(senseval2_keys)
senseval3_test_ws = get_senses(senseval3_keys)
semcor_test_ws = get_senses(semcortest_keys)

# --> leverages new ws_encoder package and added functions to utils.py
#build vocab of word sense assignments
# --> Use TRAINING data to build the vocab
### --- WILL REFERENCE VOCAB AND INDEX THROUGHOUT DOCUMENT --- ###

ws_vocab = utils.build_ws_encodings(train_word_senses)
ws_index = ws_vocab.id_to_word_sense
ws_reverse_index = ws_vocab.word_sense_to_id

# # print counts and set of word senses :
print "WS ENCODINGS: WORDSET EXAMPLES", list(itertools.islice(ws_vocab.wordset, 0, 10))
print 
print "WS ENCODINGS: UNIGRAM COUNTS EXAMPLES", list(itertools.islice(ws_vocab.unigram_counts, 0, 10))
print
print "WS ENCODINGS: INDEX EXAMPLES", list(itertools.islice(ws_index.items(), 0, 10))
print
print "WS ENCODINGS: REVERSE INDEX EXAMPLES", list(itertools.islice(ws_reverse_index.items(), 0, 10))
print

#get train and test ids only of train/test words (ordered)
train_ids = utils.preprocess_word_senses(train_word_senses, ws_vocab)
semeval2007_test_ids = utils.preprocess_word_senses(semeval2007_test_ws, ws_vocab)
semeval2013_test_ids = utils.preprocess_word_senses(semeval2013_test_ws, ws_vocab)
semeval2015_test_ids = utils.preprocess_word_senses(semeval2015_test_ws, ws_vocab)
senseval2_test_ids = utils.preprocess_word_senses(senseval2_test_ws, ws_vocab)
senseval3_test_ids = utils.preprocess_word_senses(senseval3_test_ws, ws_vocab)
semcor_test_ids = utils.preprocess_word_senses(semcor_test_ws, ws_vocab)

#show example of train_ids
print "TRAIN IDS EXAMPLES:",train_ids[0:100]
print 
print "TEST IDS EXAMPLES:",semcor_test_ids[0:100]
print
print "SEMEVAL2007 TEST IDS EXAMPLES:",semeval2007_test_ids[0:10]


### MAKE MAP OF INSTANCE ID -> WORD SENSE/WORD SENSE INDEX

In [ ]:
"""ADDED 8_13_17"""

from collections import defaultdict

def lookup_index(word_sense, reverse_index) :
    #looks up a word sense index number from a word sense
    ws_index = None
    try :
        ws_index = reverse_index[word_sense]
    except :
        ws_index = 0 #reserved index number for unk
    return ws_index

def make_keys_dict(keys) :
    #returns a dict of instance ID -> word sense
    ws_map = dict()
    for key in keys:
        ID, word_sense = key.split(" ")
        ws_map[ID] = word_sense.rstrip()
    return ws_map        

def make_indexed_keys_dict(keys, reverse_index) :
    #returns a dict of instance ID -> word sense index number
    ws_map_indexed = dict()
    for key in keys:
        ID, word_sense = key.split(" ")
        ws_map_indexed[ID] = lookup_index(word_sense.rstrip(), reverse_index)
    return ws_map_indexed

#create train and test maps
train_map = make_keys_dict(train_keys)
test_map = make_keys_dict(test_keys)
semeval2007_map = make_keys_dict(semeval2007_keys)
semeval2013_map = make_keys_dict(semeval2013_keys)
semeval2015_map = make_keys_dict(semeval2015_keys)
senseval2_map = make_keys_dict(senseval2_keys)
senseval3_map = make_keys_dict(senseval3_keys)

train_map_indexed = make_indexed_keys_dict(train_keys, ws_reverse_index)
test_map_indexed = make_indexed_keys_dict(test_keys, ws_reverse_index)

print "TRAIN MAP EXAMPLES:\n", list(itertools.islice(train_map.iteritems(), 0, 10))
print 
print "TRAIN MAP *INDEXED* EXAMPLES:\n", list(itertools.islice(train_map_indexed.iteritems(), 0, 10))


### GET RELEVANT "ROOT" DATA FROM XML FILE ###

In [ ]:
#GET MAIN XML DATA - UNCOMMENT TO GET RELEVANT DATASET

#https://docs.python.org/2/library/xml.etree.elementtree.html
#alternate good resource: https://pymotw.com/2/xml/etree/ElementTree/parse.html
import xml.etree.ElementTree as ElementTree

#SEMCOR ONLY
semcor_root = ElementTree.parse('WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.data.xml').getroot()
semeval2007_root = ElementTree.parse('WSD_Evaluation_Framework/Evaluation_Datasets/semeval2007/semeval2007.data.xml').getroot()
semeval2013_root = ElementTree.parse('WSD_Evaluation_Framework/Evaluation_Datasets/semeval2013/semeval2013.data.xml').getroot()
semeval2015_root = ElementTree.parse('WSD_Evaluation_Framework/Evaluation_Datasets/semeval2015/semeval2015.data.xml').getroot()
senseval2_root = ElementTree.parse('WSD_Evaluation_Framework/Evaluation_Datasets/senseval2/senseval2.data.xml').getroot()
senseval3_root = ElementTree.parse('WSD_Evaluation_Framework/Evaluation_Datasets/senseval3/senseval3.data.xml').getroot()
#SEMCOR+OMSTI
# semcor_omsti_root = ElementTree.parse('WSD_Evaluation_Framework/Training_Corpora/SemCor+OMSTI/semcor+omsti.gold.key.txt').getroot()

###MODIFIED 8_12_17
###TRAIN/TEST SETS :
###COMMENT OUT IF NOT ALREADY CREATED
semcortrain_root = ElementTree.parse('WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.data.train.xml').getroot()
semcortest_root = ElementTree.parse('WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.data.test.xml').getroot()

#SET CURRENT ROOTS
root = semcor_root
test_root = semcortest_root
train_root = semcortrain_root

### BUILD TRAIN/TEST SPLIT OF SEMCOR DATA

In [ ]:
# """KO MODIFIED 8/13/17 - MODIFY XML ROOT TO TRAIN/TEST"""

import xml.etree.ElementTree         

root_input = 'WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.data.xml'
output_folder = "WSD_Evaluation_Framework/Training_Corpora/SemCor/"
train_name = "semcor.data.train.xml"
test_name = "semcor.data.test.xml"
train_path = output_folder+train_name
test_path = output_folder+test_name

with open(root_input, 'r') as f:
    text = f.readlines()
    header = text[0:2]
    footer = text[-1]

with open(train_path, 'wb') as f:
    with open(test_path, 'wb') as f2:
        for line in header :
            f.write(line)
            f2.write(line)
        
        curr_text_id = None
        curr_text_id_num = None
        for line in text :
            if line[0:8] == "<text id" :
                curr_text_id = line.split('"',2)[1]
                curr_text_id_num = int(curr_text_id[1:])
            if curr_text_id_num is not None :
                if curr_text_id_num %100 <> 0 :
                    f.write(line)
                else :
                    f2.write(line)
        
        for line in footer :
            #note: goes opposite logic (because footer will already be written)
            if not curr_text_id_num %100 <> 0 :
                f.write(line)
            else :
                f2.write(line)

# _LSTM MODEL:_

# Create a list for train/test that includes all words and hardcodes a sense ID for non-ambiguous terms

### Non-ambiguous terms take on the sense key: 'NA_ID'+word

In [470]:
train_words_all = list()
train_senses_all = list()
test_words_all = list()
test_senses_all = list()
semeval2007_words_all = list()
semeval2013_words_all = list()
semeval2015_words_all = list()
senseval2_words_all = list()
senseval3_words_all = list()
semeval2007_senses_all = list()
semeval2013_senses_all = list()
semeval2015_senses_all = list()
senseval2_senses_all = list()
senseval3_senses_all = list()

"""KO MODIFIED: ROOT NAMES (BUT DIDN'T CHANGE WHERE FROM)"""
"""REVIEW: MODIFY TO ALL INSTEAD OF [:2] ON FINAL TRAINING"""

#Train
train_words_all.append('<unk>')
train_senses_all.append('<unk>')
# for doc in semcortrain_root.getchildren()[:2]:
for doc in semcortrain_root.getchildren()[:80]:
    for sent in doc:
        for text in sent:
            train_words_all.append(text.attrib['lemma'].lower())
            if text.tag == 'instance':
                 train_senses_all.append(train_map[str(text.attrib['id'])].rstrip())
            else:
                train_senses_all.append("NA_ID_")
                
#Test
test_words_all.append('<unk>')
semeval2007_words_all.append('<unk>')
semeval2013_words_all.append('<unk>')
semeval2015_words_all.append('<unk>')
senseval2_words_all.append('<unk>')
senseval3_words_all.append('<unk>')

test_senses_all.append('<unk>')
semeval2007_senses_all.append('<unk>')
semeval2013_senses_all.append('<unk>')
semeval2015_senses_all.append('<unk>')
senseval2_senses_all.append('<unk>')
senseval3_senses_all.append('<unk>')

# SemCor Root
for doc in semcortest_root.getchildren():
    for sent in doc:
        for text in sent:
            test_words_all.append(text.attrib['lemma'].lower())
            
            if text.tag == 'instance':
                 test_senses_all.append(test_map[str(text.attrib['id'])].rstrip())
            else:
                test_senses_all.append("NA_ID_")                
#                 test_senses_all.append("NA_ID_"+text.attrib['lemma'].lower())

# SemEval2007  
for doc in semeval2007_root.getchildren():
    for sent in doc:
        for text in sent:
            semeval2007_words_all.append(text.attrib['lemma'].lower())
            
            if text.tag == 'instance':
                 semeval2007_senses_all.append(semeval2007_map[str(text.attrib['id'])].rstrip())
            else:
                semeval2007_senses_all.append("NA_ID_")                
            
# SemEval2013   
for doc in semeval2013_root.getchildren():
    for sent in doc:
        for text in sent:
            semeval2013_words_all.append(text.attrib['lemma'].lower())
            
            if text.tag == 'instance':
                 semeval2013_senses_all.append(semeval2013_map[str(text.attrib['id'])].rstrip())
            else:
                semeval2013_senses_all.append("NA_ID_")                

# SemEval2015               
for doc in semeval2015_root.getchildren():
    for sent in doc:
        for text in sent:
            semeval2015_words_all.append(text.attrib['lemma'].lower())
            
            if text.tag == 'instance':
                 semeval2015_senses_all.append(semeval2015_map[str(text.attrib['id'])].rstrip())
            else:
                semeval2015_senses_all.append("NA_ID_")                

# SensEval2
for doc in senseval2_root.getchildren():
    for sent in doc:
        for text in sent:
            senseval2_words_all.append(text.attrib['lemma'].lower())
            
            if text.tag == 'instance':
                 senseval2_senses_all.append(senseval2_map[str(text.attrib['id'])].rstrip())
            else:
                senseval2_senses_all.append("NA_ID_")                
            
# SensEval3
for doc in senseval3_root.getchildren():
    for sent in doc:
        for text in sent:
            senseval3_words_all.append(text.attrib['lemma'].lower())
            
            if text.tag == 'instance':
                 senseval3_senses_all.append(senseval3_map[str(text.attrib['id'])].rstrip())
            else:
                senseval3_senses_all.append("NA_ID_")                        
        
print 'Train Length'
print len(train_words_all)
print len(train_senses_all)

print '\n'+'Test Length'
print len(test_words_all)
print len(test_senses_all)


print '\n'+'Eval Word Lengths'
print 'SemEval2007\t',len(semeval2007_words_all)
print 'SemEval2013\t',len(semeval2013_words_all)
print 'SemEval2015\t',len(semeval2015_words_all)
print 'SensEval2\t',len(senseval2_words_all)
print 'SensEval3\t',len(senseval3_words_all)

print '\n'+'Eval Sense Lengths'
print 'SemEval2007\t',len(semeval2007_senses_all)
print 'SemEval2013\t',len(semeval2013_senses_all)
print 'SemEval2015\t',len(semeval2015_senses_all)
print 'SensEval2\t',len(senseval2_senses_all)
print 'SensEval3\t',len(senseval3_senses_all)

/home/cscaudill/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:23: DeprecationWarning:

This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.



Train Length
180720
180720

Test Length
9305
9305

Eval Word Lengths
SemEval2007	3202
SemEval2013	8392
SemEval2015	2605
SensEval2	5767
SensEval3	5542

Eval Sense Lengths
SemEval2007	3202
SemEval2013	8392
SemEval2015	2605
SensEval2	5767
SensEval3	5542


/home/cscaudill/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:48: DeprecationWarning:

This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.

/home/cscaudill/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:60: DeprecationWarning:

This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.

/home/cscaudill/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:71: DeprecationWarning:

This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.

/home/cscaudill/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:82: DeprecationWarning:

This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.

/home/cscaudill/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:93: DeprecationWarning:

This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.



In [ ]:
train_senses_all

In [ ]:
test_senses_all

# Convert the full train/test vocabulary to numeric IDs

In [ ]:
"""MODIFY TO ONLY INCLUDE TRAINING WORDS"""

import numpy as np

lstm_train_ids = list()
lstm_test_ids = list()
lstm_semeval2007_ids = list()
lstm_semeval2013_ids = list()
lstm_semeval2015_ids = list()
lstm_senseval2_ids = list()
lstm_senseval3_ids = list()

train_dict = {}

#Create the full vocabulary
full_words = train_words_all + test_words_all + semeval2007_words_all + semeval2013_words_all + semeval2015_words_all + senseval2_words_all + senseval3_words_all
counter=0

#generate an ID for each term and place it in a dictionary for later lookup
# train_dict['<unk>'] = counter
for i in train_words_all:
    if i not in train_dict:
        train_dict[i]=counter
        counter+=1


#create a list of IDs by looking up the ID from the dictionary
for i in train_words_all:
    lstm_train_ids.append(train_dict[i])
for j in test_words_all:
    try :
        lstm_test_ids.append(train_dict[j])
    except :
        #Not in training dictionary, assign to <unk>
        lstm_test_ids.append(0)
        
for j in semeval2007_words_all:
    try :
        lstm_semeval2007_ids.append(train_dict[j])
    except :
        #Not in training dictionary, assign to <unk>
        lstm_semeval2007_ids.append(0)

for j in semeval2013_words_all:
    try :
        lstm_semeval2013_ids.append(train_dict[j])
    except :
        #Not in training dictionary, assign to <unk>
        lstm_semeval2013_ids.append(0)
        
for j in semeval2015_words_all:
    try :
        lstm_semeval2015_ids.append(train_dict[j])
    except :
        #Not in training dictionary, assign to <unk>
        lstm_semeval2015_ids.append(0)
        
for j in senseval2_words_all:
    try :
        lstm_senseval2_ids.append(train_dict[j])
    except :
        #Not in training dictionary, assign to <unk>
        lstm_senseval2_ids.append(0)
        
for j in senseval3_words_all:
    try :
        lstm_senseval3_ids.append(train_dict[j])
    except :
        #Not in training dictionary, assign to <unk>
        lstm_senseval3_ids.append(0)
        
lstm_train_ids = np.array(lstm_train_ids)
lstm_test_ids = np.array(lstm_test_ids)
lstm_semeval2007_ids = np.array(lstm_semeval2007_ids)
lstm_semeval2013_ids = np.array(lstm_semeval2013_ids)
lstm_semeval2015_ids = np.array(lstm_semeval2015_ids)
lstm_senseval2_ids = np.array(lstm_senseval2_ids)
lstm_senseval3_ids = np.array(lstm_senseval3_ids)

print "TRAIN_WORDS_ALL EXAMPLES:", train_words_all[0:5]        
print "TRAIN_DICT EXAMPLES:", list(itertools.islice(train_dict.iteritems(), 0, 5))

print "TRAIN_IDS EXAMPLES (ACT LEMMAS):", lstm_train_ids[0:30]
print "TEST_IDS EXAMPLES (ACT LEMMAS):", lstm_test_ids[0:30]

print 'Number of Training IDs:\t',len(lstm_train_ids)
print 'Number of Test IDs:\t',len(lstm_test_ids)

inv_train_dict = {v: k for k, v in train_dict.iteritems()}
print "INV_TRAIN_DICT EXAMPLES:", list(itertools.islice(inv_train_dict.iteritems(), 0, 5))
print len(train_dict.keys())
print len(inv_train_dict.keys())


# Convert all senses and to numeric IDs

In [ ]:
train_sense_dict = {}
lstm_train_sense_ids = list()
lstm_test_sense_ids = list()
lstm_semeval2007_sense_ids = list()
lstm_semeval2013_sense_ids = list()
lstm_semeval2015_sense_ids = list()
lstm_senseval2_sense_ids = list()
lstm_senseval3_sense_ids = list()

# Create the full vocabulary
# lstm_senses = train_senses_all + test_senses_all
counter=0

#generate an ID for each term and place it in a dictionary for later lookup
# train_sense_dict['<unk>']=counter
for ws in train_senses_all:
    if ws not in train_sense_dict:
        train_sense_dict[ws]=counter
        counter+=1      

#create a list of IDs by looking up the ID from the dictionary
for train_ws in train_senses_all:
    lstm_train_sense_ids.append(train_sense_dict[train_ws])
for test_ws in test_senses_all:
    try :
        lstm_test_sense_ids.append(train_sense_dict[test_ws])
    except :
        #assign to <unk>
        lstm_test_sense_ids.append(0)

# SemEval2007
for s2007_ws in semeval2007_senses_all:
    try :
        lstm_semeval2007_sense_ids.append(train_sense_dict[s2007_ws])
    except :
        lstm_semeval2007_sense_ids.append(0)

# SemEval2013
for s2013_ws in semeval2013_senses_all:
    try :
        lstm_semeval2013_sense_ids.append(train_sense_dict[s2013_ws])
    except :
        lstm_semeval2013_sense_ids.append(0)
        
# SemEval2015
for s2015_ws in semeval2015_senses_all:
    try :
        lstm_semeval2015_sense_ids.append(train_sense_dict[s2015_ws])
    except :
        #assign to <unk>
        lstm_semeval2015_sense_ids.append(0)
        
# SensEval2
for s2_ws in senseval2_senses_all:
    try :
        lstm_senseval2_sense_ids.append(train_sense_dict[s2_ws])
    except :
        #assign to <unk>
        lstm_senseval2_sense_ids.append(0)
        
# SensEval3
for s3_ws in senseval3_senses_all:
    try :
        lstm_senseval3_sense_ids.append(train_sense_dict[s3_ws])
    except :
        #assign to <unk>
        lstm_senseval3_sense_ids.append(0)

    
lstm_train_sense_ids = np.array(lstm_train_sense_ids)
lstm_test_sense_ids = np.array(lstm_test_sense_ids)
lstm_semeval2007_sense_ids = np.array(lstm_semeval2007_sense_ids)
lstm_semeval2013_sense_ids = np.array(lstm_semeval2013_sense_ids)
lstm_semeval2015_sense_ids = np.array(lstm_semeval2015_sense_ids)
lstm_senseval2_sense_ids = np.array(lstm_senseval2_sense_ids)
lstm_senseval3_sense_ids = np.array(lstm_senseval3_sense_ids)

print "train_senses_all EXAMPLES:", train_senses_all[0:3]        
print "train_sense_dict EXAMPLES:", list(itertools.islice(train_sense_dict.iteritems(), 0, 3))

print len(test_senses_all)
print "lstm_train_sense_ids EXAMPLES (ACT SENSES):", lstm_train_sense_ids[0:20]
print "lstm_test_sense_ids (ACT SENSES - PER TRAINING):", lstm_test_sense_ids[0:20]
print "length lstm_test_sense_ids:", len(lstm_test_sense_ids)

print 'Number of Training Sense IDs:\t',len(lstm_train_sense_ids)
print 'Number of Test Sense IDs:\t',len(lstm_test_sense_ids)

inv_train_sense_dict = {v: k for k, v in train_sense_dict.iteritems()}
print "inv_train_sense_dict EXAMPLES:", list(itertools.islice(inv_train_sense_dict.iteritems(), 0, 5))

print inv_train_sense_dict[14]
print len(train_sense_dict.keys())
print len(inv_train_sense_dict.keys())

In [ ]:

#create dictionary with order_index encountered, (lemma, ws)
train_order_dict = {}
for order_index, (lemma, ws) in enumerate(zip(train_words_all,train_senses_all)) :
    train_order_dict[order_index] = lemma, ws

print len(train_words_all)
print len(train_senses_all)
    
print "TRAINING ORDER DICT: ", list(itertools.islice(train_order_dict.iteritems(), 0, 5))

#create train_word_senses_counts dictionary (used after for getting set of word senses per lemma)
train_word_senses_counts_dict = defaultdict(lambda: defaultdict(lambda: 0))
for order_index, (lemma, ws) in train_order_dict.iteritems():
    train_word_senses_counts_dict[lemma][ws]+=1
print 
print "TRAINING WORD SENSES COUNTS DICT: ", list(itertools.islice(train_word_senses_counts_dict.iteritems(), 0, 5))
    
#get set of word senses per lemma encountered
train_word_senses_dict = defaultdict(None)
for lemma, ws_counts in train_word_senses_counts_dict.iteritems() :
    train_word_senses_dict[lemma] = ws_counts.keys()
print
print "TRAINING WORD SENSES DICT: ", list(itertools.islice(train_word_senses_dict.iteritems(), 0, 5))


In [ ]:
print lstm_train_ids[:60]

In [ ]:
print lstm_train_sense_ids[:40]

# Functions to take the numeric IDs from above and lookup the corresponding word/sense

In [ ]:
"REDO AND USE ORDER_DICT AND KEYS"

def RNN_WordLookup(train_orderID):
    try :
        return train_order_dict[train_orderID][0]
    except :
        return None
          
def RNN_SenseLookup(train_orderID):   
    try :
        return train_order_dict[train_orderID][1]
    except :
        return None

      
"RNN WORD, SENSE LOOKUPS FOR A SPECIFIC ID #"
print RNN_WordLookup(45)
print RNN_SenseLookup(45)


# RNNLM

In [ ]:
import json, os, re, shutil, sys, time
import collections, itertools
import unittest
from IPython.display import display, HTML
import operator

# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import tensorflow as tf
assert(tf.__version__.startswith("1.1"))

# utils.pretty_print_matrix uses Pandas. Configure float format here.
import pandas as pd
pd.set_option('float_format', lambda f: "{0:.04f}".format(f))

# Helper libraries
from shared_lib import utils, vocabulary, tf_embed_viz, ws_encoder

# Your code
import rnnlm
reload(rnnlm)

### Show how the batches feed into LSTM

In [ ]:
html = "<h3>Input words w:</h3>"
html += "<table><tr><th>Batch 0</th><th>Batch 1</th></tr><tr>"

bi = utils.custom_batch_generator(lstm_train_ids[:40], lstm_train_sense_ids[:40], batch_size=4, max_time=5)
for i, (w,y) in enumerate(bi):
    html += "<td>" + utils.render_matrix(w, cols=["w_%d" % d for d in range(w.shape[1])], dtype=object) + "</td>"
html += "</tr></table>"
display(HTML(html))

html = "<h3>Target senses y:</h3>"
html += "<table><tr><th>Batch 0</th><th>Batch 1</th></tr><tr>"
bi = utils.custom_batch_generator(lstm_train_ids[:40], lstm_train_sense_ids[:40], batch_size=4, max_time=5)
for i, (w,y) in enumerate(bi):
    html += "<td>" + utils.render_matrix(y, cols=["y_%d" % d for d in range(y.shape[1])], dtype=object) + "</td>"
html += "</tr></table>"
display(HTML(html))

# Create RNN Training Functions

In [ ]:
def run_epoch(lm, session, batch_iterator,
              train=False, verbose=True,
              tick_s=10, learning_rate=0.1):
    start_time = time.time()
    tick_time = start_time  # for showing status
    total_cost = 0.0  # total cost, summed over all words
    total_batches = 0
    total_words = 0
    logs = list()
    sm_logs = list()
    ins = list()
    proba_list = list()
    pmax_list = list()
    prand_list = list()

    if train:
        train_op = lm.train_step_
        use_dropout = True
        loss = lm.train_loss_
    else:
        train_op = tf.no_op()
        use_dropout = False  # no dropout at test time
        loss = lm.loss_  # true loss, if train_loss is an approximation

    for i, (w, y) in enumerate(batch_iterator):
        cost = 0.0
        # At first batch in epoch, get a clean intitial state.
        if i == 0:
            h = session.run(lm.initial_h_, {lm.input_w_: w})

        #### YOUR CODE HERE ####
        feed_dict = {lm.input_w_:w,
                     lm.target_y_:y,
                     lm.initial_h_:h,
                     lm.learning_rate_:learning_rate,
                     lm.use_dropout_:use_dropout}
        
        ### KO MODIFIED
        if not train :
            h, cost,sm_logits,_ = session.run([lm.final_h_, loss, lm.softmax_logits_, train_op], feed_dict=feed_dict)          
        else :
            h, cost,_ = session.run([lm.final_h_, loss, train_op], feed_dict=feed_dict)
        
        total_cost += cost
        total_batches = i + 1
        total_words += w.size  # w.size = batch_size * max_time
        
        #KO MODIFIED 
        if not train :
#             logs.append(logits)
            sm_logs.append(sm_logits)
        
        ##
        # Print average loss-so-far for epoch
        # If using train_loss_, this may be an underestimate.
        if verbose and (time.time() - tick_time >= tick_s):
            avg_cost = total_cost / total_batches
            avg_wps = total_words / (time.time() - start_time)
            print "[batch %d]: seen %d words at %d wps, loss = %.3f" % (
                i, total_words, avg_wps, avg_cost)
            tick_time = time.time()  # reset time ticker
            
    #KO MODIFIED
    if not train :
        return (total_cost / total_batches), sm_logs
    else :
        return (total_cost / total_batches)       

In [ ]:
def score_dataset(lm, session, word_ids,sense_ids, name="Data"):
    # For scoring, we can use larger batches to speed things up.
    bi = utils.custom_batch_generator(word_ids, sense_ids, batch_size=100, max_time=100)
    cost = run_epoch(lm, session, bi, 
                     learning_rate=1.0, train=False, 
                     verbose=False, tick_s=3600)[0]
    print "%s: avg. loss: %.03f  (perplexity: %.02f)" % (name, cost, np.exp(cost))

# Define LSTM Training Parameters

In [ ]:
# Training parameters
max_time = 100
batch_size = 30
learning_rate = 0.2
### MODIFIED to go faster
num_epochs = 10
# num_epochs = 30

# Model parameters
model_params = dict(V=len(train_senses_all),
                    H=400, 
                    softmax_ns=200,
                    num_layers=4)

TF_SAVEDIR = "tf_saved"
checkpoint_filename = os.path.join(TF_SAVEDIR, "rnnlm")
trained_filename = os.path.join(TF_SAVEDIR, "rnnlm_trained")

# Execute Training

In [ ]:
reload(rnnlm)
# Will print status every this many seconds
print_interval = 5

# Clear old log directory
shutil.rmtree("tf_summaries", ignore_errors=True)

lm = rnnlm.RNNLM(**model_params)
lm.BuildCoreGraph()
lm.BuildTrainGraph()
lm.BuildSamplerGraph()

# Explicitly add global initializer and variable saver to LM graph
with lm.graph.as_default():
    initializer = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
# Clear old log directory
shutil.rmtree(TF_SAVEDIR, ignore_errors=True)
if not os.path.isdir(TF_SAVEDIR):
    os.makedirs(TF_SAVEDIR)

with tf.Session(graph=lm.graph) as session:
    # Seed RNG for repeatability
    tf.set_random_seed(42)

    session.run(initializer)

    for epoch in xrange(1,num_epochs+1):
        t0_epoch = time.time()
        bi = utils.custom_batch_generator(lstm_train_ids, lstm_train_sense_ids, batch_size, max_time)
        print "[epoch %d] Starting epoch %d" % (epoch, epoch)

        # Run a training epoch.
        ### KO MODIFIED: REMOVE LOGITS
        cost = run_epoch(lm,session,bi,train=True, learning_rate=learning_rate)
        
        print "[epoch %d] Completed in %s" % (epoch, utils.pretty_timedelta(since=t0_epoch))
    
        # Save a checkpoint
        saver.save(session, checkpoint_filename, global_step=epoch)
        
#         print ("[epoch %d]" % epoch),
#         score_dataset(lm, session, lstm_train_ids,lstm_train_sense_ids, name="Train set")
#         print ("[epoch %d]" % epoch),
#         score_dataset(lm, session, lstm_test_ids, lstm_test_sense_ids, name="Test set")
#         print ""
    
    # Save final model

    saver.save(session, trained_filename)

# Run on Test data

## In the cell below, remove the comments for the evaluation set that you'd like to test with.

In [ ]:
print len(senseval3_test_ids)
print len(lstm_senseval3_ids)
print len(lstm_senseval3_sense_ids)
print len(senseval3_words_all)
print len(senseval3_senses_all)
print len(senseval3_root)

In [457]:
def selectTestData(data):

    if data == 'SemCorTest':
        return semcor_test_ids,lstm_test_ids,lstm_test_sense_ids,test_words_all,test_senses_all,semcortest_root

    elif data == 'SemEval2007':
        return semeval2007_test_ids,lstm_semeval2007_ids,lstm_semeval2007_sense_ids,semeval2007_words_all,semeval2007_senses_all,semeval2007_root

    elif data == 'SemEval2013':
        return semeval2013_test_ids,lstm_semeval2013_ids,lstm_semeval2013_sense_ids,semeval2013_words_all,semeval2013_senses_all,semeval2013_root

    elif data == 'SemEval2015': 
        return semeval2015_test_ids,lstm_semeval2015_ids,lstm_semeval2015_sense_ids,semeval2015_words_all,semeval2015_senses_all,semeval2015_root

    elif data == 'SensEval2':
        return senseval2_test_ids,lstm_senseval2_ids,lstm_senseval2_sense_ids,senseval2_words_all,senseval2_senses_all,senseval2_root

    elif data == 'SensEval3':
        return senseval3_test_ids,lstm_senseval3_ids,lstm_senseval3_sense_ids,senseval3_words_all,senseval3_senses_all,senseval3_root
    

var_test_ids,var_lstm_test_ids,var_test_sense_ids,var_test_words_all,var_test_senses_all,var_test_root = selectTestData('SemCorTest')



"""CREATE ORDER DICT FOR EVERY TEST/EVAL SET TO CREATE PREDICTIVE OUTPUT IN LSTM"""
test_order_dict = {}
for order_index, (lemma, ws) in enumerate(zip(var_test_words_all,var_test_senses_all)) :
    test_order_dict[order_index] = lemma, ws

print len(var_test_ids)

2532


In [436]:
lm = rnnlm.RNNLM(**model_params)
lm.BuildCoreGraph()
# # lm.BuildTrainGraph()
lm.BuildSamplerGraph()

# Explicitly add global initializer and variable saver to LM graph
with lm.graph.as_default():
    initializer = tf.global_variables_initializer()
    saver = tf.train.Saver()

with tf.Session(graph=lm.graph) as session:
    # Seed RNG for repeatability
    tf.set_random_seed(42)

    saver.restore(session, trained_filename)

    t0_epoch = time.time()
    bi = utils.custom_batch_generator(var_lstm_test_ids, var_test_sense_ids, batch_size, max_time)
    print "Starting epoch"
    # Run a training epoch.
    """KO MODIFIED TO _TEST"""
    cost, sm_logits_test = run_epoch(lm,session,bi,train=False, learning_rate=learning_rate)
    print "Completed in %s" % (utils.pretty_timedelta(since=t0_epoch))
    
#     print ("[epoch %d]" % epoch),
#     score_dataset(lm, session, lstm_train_ids, lstm_train_sense_ids, name="Train set")
    score_dataset(lm, session, var_lstm_test_ids, var_test_sense_ids, name="Test set")
    print ""


INFO:tensorflow:Restoring parameters from tf_saved/rnnlm_trained
Starting epoch
Completed in 0:00:08
Test set: avg. loss: 3.136  (perplexity: 23.01)



In [437]:
def returnPossibleSenses(word) :
    #from a word like "make" look up all possible word senses
    try :
        return train_word_senses_dict[word]
    except :
        return None

def returnWordSenseIDs(word_senses) :
    #returns all WORD SENSE IDs for list of word senses ['make%2:36:00::\n', 'make%2:31:13::\n']
    word_sense_IDs = list()
    for word_sense in word_senses :
        
        if word_sense in train_sense_dict :
            word_sense_IDs.append(train_sense_dict[word_sense])
        else :
            continue
    word_sense_IDs = set(word_sense_IDs)
    return word_sense_IDs

print "POSSIBLE SENSES:"
print returnPossibleSenses('how')
print returnPossibleSenses('make')
print returnPossibleSenses('qwerty')

print "WORD SENSE IDs:"
print returnWordSenseIDs(['make%2:36:00::', 'make%2:31:13::', 'make%2:41:00::', 'make%2:42:00::', 'make%2:30:00::', 'NA_ID_make', 'NA_ID_made'])
print returnWordSenseIDs(['qwerty'])

POSSIBLE SENSES:
['NA_ID_']
['make%2:36:08::', 'make%2:29:08::', 'make%2:36:11::', 'make%2:41:00::', 'make%2:36:13::', 'make%2:30:00::', 'make%2:31:13::', 'make%2:36:15::', 'make%2:41:03::', 'make%2:30:02::', 'make%2:38:00::', 'make%2:32:00::', 'make%2:36:01::', 'make%2:42:05::', 'make%2:36:00::', 'make%2:42:00::', 'make%2:31:00::', 'make%2:40:02::', 'make%2:40:01::', 'make%1:09:00::', 'make%2:38:02::', 'make%2:36:09::', 'NA_ID_']
None
WORD SENSE IDs:
set([82, 339, 324, 69, 29])
set([])


# Creating a dictionary of logits for each word

In [438]:

order_index=0

## Logit data is output in many embedded arrays
## Number of terms in each back is equal to -> (max_time * batch_length)

print order_index


#MODIFY ORDERED_DICT AS APPROPRIATE
def return_sense_id_preds(logits, possible_senses) :

    preds_dict = None
    if possible_senses is None :
        return preds_dict
        
    word_sense_indices = []
    for word_sense in possible_senses :
        
        #exclude NA_IDs (non-instances) from normalization
        if word_sense[0:5] == "NA_ID" :
            continue
        else :
            #get word sense index, append to possible indices
            try :
                word_sense_index = train_sense_dict[word_sense]
                word_sense_indices.append(word_sense_index)
            except :
                continue
    
    total = 0
    
    
    for word_sense_index in word_sense_indices :
        try :
            total += logits[word_sense_index]
        except :
            continue
    #normalize
    try :
        normalizer = 1/float(total)
    except :
        normalizer = 1
    
    #EDITED 8_19_17_KO 
    if word_sense_indices <> [] :
        preds_dict = {}

        #normalize
        for word_sense_index in word_sense_indices :
            preds_dict[inv_train_sense_dict[word_sense_index]] = logits[word_sense_index]*normalizer
    
#     print preds_dict
    return preds_dict

# print return_sense_id_preds(sm_logits_test[0][0][1], 
#                             returnPossibleSenses(test_order_dict[7][0]))
        

"""COPY/PASTE FOR EVAL SETS AS APPROPRIATE
sm_logits_dict_test
sm_logits_test (shows twice)
test_order_dict

"""

sm_logits_dict_test = defaultdict(lambda: defaultdict(lambda: 0))


batch_num = 0
index_id = 0
for batch in sm_logits_test:
    word_num = 0
    for word in batch:
        log_num = 0
        for logs in word:
            sm_logits_dict_test[index_id] = return_sense_id_preds(logits=sm_logits_test[batch_num][word_num][log_num],
                                                 possible_senses=returnPossibleSenses(test_order_dict[index_id][0]))
            
#             if index_id == 1 :
#                 print "LOGITS:", sm_logits_test[batch_num][word_num][log_num]
#                 print "POSSIBLE SENSES:", returnPossibleSenses(test_order_dict[index_id][1])
#                 print "RETURN PREDS DICT: ", return_sense_id_preds(sm_logits_test[batch_num][word_num][log_num],
#                                                                   returnPossibleSenses(test_order_dict[index_id][0]))

            index_id += 1
            log_num += 1
        word_num += 1
    batch_num += 1

print "INDEX_ID", index_id
print "LOGITS TEST DICT EXAMPLES:", sm_logits_dict_test[5]

# print "LOGITS TEST DICT EXAMPLES:", list(itertools.islice(sm_logits_dict_test.iteritems(), 2, 5))

0
INDEX_ID 5520
LOGITS TEST DICT EXAMPLES: {'man%1:18:04::': 0.011846504099500939, 'man%1:05:01::': 2.3607048656445064e-07, 'man%1:18:08::': 2.3239614571941497e-07, 'man%1:18:03::': 0.0023169844453728695, 'man%1:18:00::': 0.98583604298849403}


# Calculate Accuracy when predicting within only the available senses from a given word

In [439]:
import operator
counter = 0
correct = 0

correct_index_list = []
correct_word_sense_list = []

print len(sm_logits_dict_test.keys())
print len(var_test_sense_ids)
print var_test_sense_ids

#note the OFFSET[:1] in lstm_test_sense_ids, as index 0 is a manually-inserted '<unk>'.
#May have to continue this in other combination sections as well
for (k, preds), act_word_sense_index in zip(sm_logits_dict_test.iteritems(), var_test_sense_ids) :
    counter += 1
    #get maximum probability, maximum word_sense (sorted makes default to first index if there is a tie)
    try :
        max_word_sense = max(sorted(preds.iteritems()), key=operator.itemgetter(1))[0]
        max_word_sense_index = train_sense_dict[max_word_sense]
        if max_word_sense_index == act_word_sense_index :
            correct += 1
            correct_index_list.append(max_word_sense_index)
            correct_word_sense_list.append(max_word_sense)
    
        else :
            pass
    except : #handle empty sets
        continue
        
print counter
print correct

print "% CORRECT (INTENTIONALLY MARKS ALL NON-INSTANCE IDs AS WRONG, SO LOW SCORE:", float(correct)/counter

5520
5542
[0 1 1 ..., 1 1 1]
5520
613
% CORRECT (INTENTIONALLY MARKS ALL NON-INSTANCE IDs AS WRONG, SO LOW SCORE: 0.111050724638


In [440]:
counter = 0
for i in correct_word_sense_list :
    if i[0:5] <> 'NA_ID' :
        counter += 1
        
print counter

613


### GET INSTANCE_IDS for TEST

In [441]:
counter = 1  #starts at 1 to skip "<unk>" at index 0
lstm_dense_preds_test = defaultdict(lambda: defaultdict(lambda: 0))

### NOTE: THIS IS ON [0:2] TO MATCH UP TO WHAT TEST OUTPUT WAS; MODIFY TO FULL SET AS APPROPRIATE

instance_counter = 0
for doc in var_test_root.getchildren()[:]:
    for sent in doc:
        for text in sent:
            if text.tag == 'instance':
                instance_counter += 1
                lstm_dense_preds_test[str(text.attrib['id'])] = sm_logits_dict_test[counter]
#                 print str(text.attrib['id']),str(text.attrib['lemma']), counter, sm_logits_dict_test[counter]
            else :
                pass            
            counter += 1

# print instance_counter
print "LSTM TEST OUTPUT (PREDS_DENSE):", list(itertools.islice(lstm_dense_preds_test.iteritems(), 0, 5))            

LSTM TEST OUTPUT (PREDS_DENSE): [('d002.s124.t001', {'house%1:04:00::': 0.59613375942387647, 'house%2:42:02::': 0.12096509549458763, 'house%2:41:00::': 0.1776297143996825, 'house%1:06:00::': 0.10527143068185335}), ('d002.s034.t000', {'huge%5:00:01:large:00': 1.0}), ('d002.s034.t001', {'fire%2:33:01::': 0.045921063652148597, 'fire%1:22:00::': 0.0045605937990364515, 'fire%1:11:00::': 0.12780875284691645, 'fire%1:04:00::': 0.022272275143687039, 'fire%2:41:00::': 0.060677808526567699, 'fire%2:33:00::': 0.73875950603164364}), ('d002.s034.t002', None), ('d001.s027.t007', {'work%1:06:01::': 0.094509294376201217, 'work%2:41:00::': 0.037240508042546513, 'work%2:41:03::': 0.019445928564722718, 'work%2:35:02::': 0.00098776325423003525, 'work%1:04:00::': 0.16742356173123224, 'work%1:06:00::': 0.0013224919830759324, 'work%2:41:04::': 0.011835364019038633, 'work%2:41:02::': 0.024488503679619439, 'work%1:04:01::': 0.012996645297749096, 'work%1:09:00::': 0.60609318447394034, 'work%2:36:00::': 0.008155

/home/cscaudill/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: DeprecationWarning:

This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.



In [442]:
with open ("lstm_output.txt", "w") as f :
    for line in sorted(lstm_dense_preds_test.iteritems()) :
        f.write(str(line))

# LSTM Conversion to Ensemble Vocab Indexes

In [443]:
###ADDED 8_19_17_KO

def lstm_sense_dict_to_ensemble_index(lstm_dict, reverse_index) :
    print  list(itertools.islice(lstm_dict.iteritems(), 0, 5)) 
    lstm_dict_index = defaultdict(dict)
    for ID, preds in lstm_dict.iteritems() :
        if preds is None or len(preds.keys()) == 0 :
            lstm_dict_index[ID] = None
        else :
            for ws, prob in preds.iteritems() :
                ws_index = lookup_index(ws, reverse_index)
                lstm_dict_index[ID][ws_index] = prob
    return lstm_dict_index

lstm_dense_preds_test_matched_index = lstm_sense_dict_to_ensemble_index(lstm_dict=lstm_dense_preds_test,
                                                                       reverse_index=ws_reverse_index)
  
print list(itertools.islice(lstm_dense_preds_test_matched_index.iteritems(), 0, 10))  

# for k in lstm_dense_preds_test.iterkeys() :
#     if k not in lstm_dense_preds_test_matched_index.keys() :
#         print k, lstm_dense_preds_test[k]

print len(lstm_dense_preds_test_matched_index.keys())

[('d002.s124.t001', {'house%1:04:00::': 0.59613375942387647, 'house%2:42:02::': 0.12096509549458763, 'house%2:41:00::': 0.1776297143996825, 'house%1:06:00::': 0.10527143068185335}), ('d002.s034.t000', {'huge%5:00:01:large:00': 1.0}), ('d002.s034.t001', {'fire%2:33:01::': 0.045921063652148597, 'fire%1:22:00::': 0.0045605937990364515, 'fire%1:11:00::': 0.12780875284691645, 'fire%1:04:00::': 0.022272275143687039, 'fire%2:41:00::': 0.060677808526567699, 'fire%2:33:00::': 0.73875950603164364}), ('d002.s034.t002', None), ('d001.s027.t007', {'work%1:06:01::': 0.094509294376201217, 'work%2:41:00::': 0.037240508042546513, 'work%2:41:03::': 0.019445928564722718, 'work%2:35:02::': 0.00098776325423003525, 'work%1:04:00::': 0.16742356173123224, 'work%1:06:00::': 0.0013224919830759324, 'work%2:41:04::': 0.011835364019038633, 'work%2:41:02::': 0.024488503679619439, 'work%1:04:01::': 0.012996645297749096, 'work%1:09:00::': 0.60609318447394034, 'work%2:36:00::': 0.0081557106197149599, 'work%2:41:05::':

# _MAKE BIGRAM/TRIGRAM MODELS:_

### CREATE CONTEXT COUNTS
##### NOTE:
 - INCLUDING CURRENT LEMMA IN CONTEXT COUNT, AS WE WANT TO LIMIT TO *ONLY* WORD SENSES FOR THAT LEMMA
 - CONTEXT INFORMATION ACTUALLY INCLUDES CURRENT LEMMA/POS INFO SINCE WE ARE GIVEN THIS IN TEXT AND WANT TO PREDICT WORD SENSE (*NOT* NEXT-WORD PREDICTION)

In [444]:
def create_ngram_counts(root, keys_map, pos_grams="N") :
    #creates unigrams in format [(P_0)][word_sense] to counts
    #creates bigrams in format [(P_1, P_0)][word sense] to counts
    #creates trigrams in format [(P_2, P_1, P_0)][word sense] to counts
    #where:
    #P_2, P_1, and P_0 can be either:
    # lemmas --> pos_grams="Y"
    # POS --> pos_grams="N"
    
    #returns unigrams, bigrams, trigrams
    
    """NOTE: Use an indexed keys map to return word sense indexes instead of word senses"""
    
    ### MODIFY TO and "Y" add reverse index to get word sense indexes instead
    """NOTE: n-grams are different than in lectures because WANT to use current context like POS and lemma;
    not just preceding context. This is because we are trying to predict the current WORD SENSE, and already
    have the current POS and lemma (due to Stanford NLP prediction used)"""
    unigram_counts = defaultdict(lambda: defaultdict(lambda: 0.0))
    bigram_counts = defaultdict(lambda: defaultdict(lambda: 0.0))
    trigram_counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    #iterate through docs
    for i in root.getchildren():
        
        #iterate through sentences
        #if at start of data, no p1/p2
        p_1 = None
        p_2 = None
        instance_id = None
        for j in i.getchildren():
            #iterate through tags
            for k in j.getchildren():
                if pos_grams == "Y" :
                    #use POS for bi/tri grams
                    p_0 = k.attrib.get('pos')
                else :
                    #use lemma for bi/tri-grams
                    p_0 = k.attrib.get('lemma')
                if k.tag == 'instance':                
                    instance_id = k.attrib.get('id')
                    word_sense = keys_map[instance_id]
                    if pos_grams == "Y" :
                        lemma = k.attrib.get('lemma')
                        #NOTE: Need to add lemmas here because need to limit to POS in just that lemma
                        unigram_counts[(p_0,lemma)][word_sense] += 1
                        bigram_counts[(p_1,p_0,lemma)][word_sense] += 1
                        trigram_counts[(p_2,p_1,p_0,lemma)][word_sense] += 1
                        
                    else :
                        unigram_counts[(p_0)][word_sense] += 1
                        bigram_counts[(p_1,p_0)][word_sense] += 1
                        trigram_counts[(p_2,p_1,p_0)][word_sense] += 1
                #set lemmas = prior lemmas
                p_2 = p_1
                p_1 = p_0
    
    return unigram_counts, bigram_counts, trigram_counts

#NOTE: using indexed version of maps
train_unigrams, train_bigrams, train_trigrams = create_ngram_counts(
    semcortrain_root, train_map_indexed, pos_grams="N")

train_unigrams_POS, train_bigrams_POS, train_trigrams_POS = create_ngram_counts(
    semcortrain_root, train_map_indexed, pos_grams="Y")

print "UNIGRAMS (lemmas):", list(itertools.islice(train_unigrams.iteritems(), 0, 5))
print
print "BIGRAMS (lemmas):", list(itertools.islice(train_bigrams.iteritems(), 0, 5))
print
print "TRIGRAMS (lemmas):", list(itertools.islice(train_trigrams.iteritems(), 0, 5))
print 
print "UNIGRAMS (POS|curr lemma):", list(itertools.islice(train_unigrams_POS.iteritems(), 0, 5))
print
print "BIGRAMS (POS|curr lemma):", list(itertools.islice(train_bigrams_POS.iteritems(), 0, 5))
print
print "TRIGRAMS (POS|curr lemma):", list(itertools.islice(train_trigrams_POS.iteritems(), 0, 5))

/home/cscaudill/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:23: DeprecationWarning:

This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.

/home/cscaudill/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:30: DeprecationWarning:

This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.

/home/cscaudill/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:32: DeprecationWarning:

This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.



UNIGRAMS (lemmas): [('fawn', defaultdict(<function <lambda> at 0x7f5b9da8a488>, {27399: 1.0})), ('xylem', defaultdict(<function <lambda> at 0x7f5bf98e07d0>, {9287: 4.0})), ('unscientific', defaultdict(<function <lambda> at 0x7f4f35f0d668>, {15671: 2.0})), ('writings', defaultdict(<function <lambda> at 0x7f4f34c17668>, {26850: 1.0})), ('systematic', defaultdict(<function <lambda> at 0x7f4f39df2cf8>, {16728: 2.0}))]

BIGRAMS (lemmas): [(('helium', 'temperature'), defaultdict(<function <lambda> at 0x7f4f4b7c8e60>, {221: 1.0})), (('little', 'note'), defaultdict(<function <lambda> at 0x7f5bf81c7b18>, {3058: 1.0})), (('recently', 'return'), defaultdict(<function <lambda> at 0x7f5bd6c7fc08>, {187: 1.0})), (('man', 'receive'), defaultdict(<function <lambda> at 0x7f5b4dcf7ed8>, {142: 1.0})), (('already', 'pupate'), defaultdict(<function <lambda> at 0x7f4f3ae00938>, {14571: 1.0}))]

TRIGRAMS (lemmas): [(('at', 'the', 'beginning'), defaultdict(<function <lambda> at 0x7f5b7bf0fc08>, {6299: 3.0, 22

### NORMALIZE CONTEXT COUNTS TO PROBABILITIES
##### NOTE:
 - WE NEED TO CREATE "BACKUPS" FOR WHEN A CONTEXT|LEMMA OR A LEMMA HAS NOT BEEN SEEN IN TEST SET
   - BACKUP FOR CONTEXT|LEMMA NOT SEEN IS EQUALLY WEIGHTED WORD SENSES FOR THAT LEMMA
   - BACKUP FOR LEMMA NOT SEEN IS EQUALLY WEIGHTED *ALL* WORD SENSES (IN NEXT CODE BLOCK)


In [445]:
### THIS SECTION TAKES COUNTS AND NORMALIZES TO PROBABILITIES (SUM to 1.0)

def normalize_counter(c):
    """Given a dictionary of <item, counts>, return <item, fraction>."""
    total = sum(c.itervalues())
    return {w:float(c[w])/total for w in c}

def normalize_gram_counts(gram_counts_dict) :
    probas = defaultdict(lambda: defaultdict(lambda: 0.0))
    for context, ctr in gram_counts_dict.iteritems():
        probas[context] = normalize_counter(ctr)
    return probas

def normalize_counter_equal_prob(c):
    """Given a dictionary of <item, counts>, return <item, fraction>."""
    average = 1/float(len(list(c.itervalues())))
    return {w:average for w in c}

def normalize_gram_counts_equal_prob(gram_counts_dict) :
    probas = defaultdict(lambda: defaultdict(lambda: 0.0))
    for context, ctr in gram_counts_dict.iteritems():
        probas[context] = normalize_counter_equal_prob(ctr)
    return probas


#GET ALL NORMALIZED UNIGRAMS
normalized_unigrams = normalize_gram_counts(train_unigrams)
normalized_bigrams = normalize_gram_counts(train_bigrams)
normalized_trigrams = normalize_gram_counts(train_trigrams)
normalized_unigrams_POS = normalize_gram_counts(train_unigrams_POS)
normalized_bigrams_POS = normalize_gram_counts(train_bigrams_POS)
normalized_trigrams_POS = normalize_gram_counts(train_trigrams_POS)

#GET NORMALIZED UNIGRAMS WITH EQUAL PROB - THIS BECOMES THE BACKUP FOR WHEN A CONTEXT ISN'T FOUND FOR A LEMMA
normalized_unigrams_equal_prob = normalize_gram_counts_equal_prob(train_unigrams)


print "NORMALIZED UNIGRAMS:", list(itertools.islice(normalized_unigrams.iteritems(), 0, 3))
print 
print "NORMALIZED BIGRAMS:", list(itertools.islice(normalized_bigrams.iteritems(), 0, 3))
print 
print "NORMALIZED TRIGRAMS:", list(itertools.islice(normalized_trigrams.iteritems(), 0, 3))
print 
print "NORMALIZED UNIGRAMS (POS):", list(itertools.islice(normalized_unigrams_POS.iteritems(), 0, 3))
print 
print "NORMALIZED BIGRAMS (POS):", list(itertools.islice(normalized_bigrams_POS.iteritems(), 0, 3))
print 
print "NORMALIZED TRIGRAMS (POS):", list(itertools.islice(normalized_trigrams_POS.iteritems(), 0, 3))
print 
print "NORMALIZED UNIGRAMS (EQUAL PROBS, USED WHEN NEW CONTEXT SEEN FOR LEMMA):", list(itertools.islice(normalized_unigrams_equal_prob.iteritems(), 0, 10))

NORMALIZED UNIGRAMS: [('fawn', {27399: 1.0}), ('xylem', {9287: 1.0}), ('unscientific', {15671: 1.0})]

NORMALIZED BIGRAMS: [(('helium', 'temperature'), {221: 1.0}), (('little', 'note'), {3058: 1.0}), (('man', 'receive'), {142: 1.0})]

NORMALIZED TRIGRAMS: [(('worthy', 'of', 'consideration'), {2292: 1.0}), (('at', 'the', 'beginning'), {6299: 0.375, 2269: 0.5, 4662: 0.125}), (('devout', ',', 'orthodox'), {10035: 1.0})]

NORMALIZED UNIGRAMS (POS): [(('VERB', 'buck_up'), {33118: 1.0}), (('NOUN', 'starting_point'), {13345: 1.0}), (('VERB', 'breathe'), {2217: 0.8947368421052632, 28291: 0.05263157894736842, 29133: 0.05263157894736842})]

NORMALIZED BIGRAMS (POS): [(('DET', 'NOUN', 'shock_therapy'), {29750: 1.0}), (('DET', 'NOUN', 'hyacinth'), {19777: 1.0}), (('ADV', 'VERB', 'look_like'), {454: 1.0})]

NORMALIZED TRIGRAMS (POS): [(('NOUN', '.', 'NOUN', 'automobile'), {2496: 1.0}), (('DET', 'DET', 'ADJ', 'public'), {612: 1.0}), (('PRT', 'DET', 'ADJ', 'single'), {333: 1.0})]

NORMALIZED UNIGRAMS

### CREATE BACKUP PROBABILITIES (ASSIGNED TO *ALL* WORD SENSES) FOR WHEN AN UNSEEN LEMMA IS ENCOUNTERED

In [446]:
#create even probabilities for ALL Word senses (per training data); used when lemma never encountered before in test/eval

def create_all_equal_probas(ws_index) :
    #feed in training index to get all equal probability output (used when NEW LEMMA)
    #do not include <unk> index prob
    len_index = len(ws_index.keys())
    backup_index_probs = dict()
    backup_word_sense_probs = dict()
    for index, word_sense in zip(ws_index.iterkeys(),ws_index.itervalues()) :
        if index not in ('<unk>',0) :
            backup_index_probs[index] = 1/float(len_index-1)
            backup_word_sense_probs[word_sense] = 1/float(len_index-1)
    return backup_index_probs, backup_word_sense_probs
    
backup_index_probs, backup_word_sense_probs = create_all_equal_probas(ws_index)

print "BACKUP INDEX PROBS (USED WHEN NEW LEMMA ENCOUNTERED):", list(itertools.islice(backup_index_probs.iteritems(), 0, 5))
print 
print "BACKUP WORD SENSE PROBS (USED WHEN NEW LEMMA ENCOUNTERED):", list(itertools.islice(backup_word_sense_probs.iteritems(), 0, 5))

BACKUP INDEX PROBS (USED WHEN NEW LEMMA ENCOUNTERED): [(1, 3.018776791644026e-05), (2, 3.018776791644026e-05), (3, 3.018776791644026e-05), (4, 3.018776791644026e-05), (5, 3.018776791644026e-05)]

BACKUP WORD SENSE PROBS (USED WHEN NEW LEMMA ENCOUNTERED): [('trumpet_vine%1:20:01::', 3.018776791644026e-05), ('movement%1:04:03::', 3.018776791644026e-05), ('scale%1:24:01::', 3.018776791644026e-05), ('b%1:05:00::', 3.018776791644026e-05), ('vacancy%1:26:00::', 3.018776791644026e-05)]


### CREATE WORD SENSE PROBABILITY PREDICTOR (GIVEN CONTEXT)

In [447]:
def ws_probas(lemma,context, n, normalized_ngrams, normalized_unigrams_equal_prob,backup_index,POS='N'):
    ###NOTE: ALWAYS FEED IN normalized_unigrams with equal_prob (non-POS) as last argument
    ###NOTE: USE THE PROPER BACKUP (index vs non-index) depending on what using
    """Compute p(each word sense | context)"""
    #convert context as necessary
        
    #return possibilities if exists
    if context in normalized_ngrams :
        return normalized_ngrams[context]    
    else :
        #IF HAVE SEEN LEMMA, return an average across all word sense instances for that lemma
        if lemma in normalized_unigrams_equal_prob :
            return normalized_unigrams_equal_prob[lemma]
        #IF HAVE NOT SEEN LEMMA, return an average across ALL WORD SENSES
        else :
            return backup_index

###try a few instances
#should return relevant context
probas_tester_1 = ws_probas(lemma='art', context=('ADJ','NOUN', 'art'), n=2, 
          normalized_ngrams=normalized_bigrams_POS, 
          normalized_unigrams_equal_prob=normalized_unigrams_equal_prob,
          backup_index = backup_index_probs,
          POS='Y')

#should default to averages; no relevant context
probas_tester_2 = ws_probas(lemma='art', context=('DET','DET', 'art'), n=2, 
          normalized_ngrams=normalized_bigrams_POS, 
          normalized_unigrams_equal_prob=normalized_unigrams_equal_prob,
          backup_index = backup_index_probs,
          POS='Y')

#should return relevant context
probas_tester_3 = ws_probas(lemma='be', context=('like', 'eggs', 'be'), n=3, 
          normalized_ngrams=normalized_trigrams, 
          normalized_unigrams_equal_prob=normalized_unigrams_equal_prob,
          backup_index = backup_index_probs,
          POS='N')

#should return relevant context
probas_tester_4 = ws_probas(lemma='iron', context=('iron'), n=1, 
          normalized_ngrams=normalized_unigrams, 
          normalized_unigrams_equal_prob=normalized_unigrams_equal_prob,
          backup_index = backup_index_probs,
          POS='N')


#should default backup_index; no relevant lemma (don't want <unk> or will skew)
probas_tester_5 = ws_probas(lemma='fhqwhgads', context=('DET','DET','fhqwhgads'), n=2, 
          normalized_ngrams=normalized_bigrams_POS, 
          normalized_unigrams_equal_prob=normalized_unigrams_equal_prob,
          backup_index = backup_index_probs,
          POS='Y')

#should return relevant context
probas_tester_6 = ws_probas(lemma='12', context=('12'), n=1, 
          normalized_ngrams=normalized_unigrams, 
          normalized_unigrams_equal_prob=normalized_unigrams_equal_prob,
          backup_index = backup_index_probs,
          POS='N')

print "Tester output: 1st and 2nd print statements should be equivalent:"
print 
print "1) WITH RELEVANT CONTEXT:"
print normalized_bigrams_POS[('ADJ','NOUN','art')]
print probas_tester_1
print
print "2) NO RELEVANT CONTEXT; SHOULD RETURN AVERAGES"
print normalized_unigrams_equal_prob[('art')]
print probas_tester_2
print
print "3) WITH RELEVANT CONTEXT:"
print normalized_trigrams[('like', 'eggs', 'be')]
print probas_tester_3
print
print "4) WITH RELEVANT CONTEXT:"
print normalized_unigrams[('iron')]
print probas_tester_4
print
print "5) NO RELEVANT LEMMA; SHOULD RETURN BACKUP (ALL) EQUAL WEIGHT"
##prints large matrix (assigns to ALL lemmas seen in training)
# print normalized_unigrams_equal_prob[('art')]
# print probas_tester_5


print "6) WITH RELEVANT CONTEXT:"
print normalized_unigrams[('12')]
print probas_tester_6


Tester output: 1st and 2nd print statements should be equivalent:

1) WITH RELEVANT CONTEXT:
{11281: 0.07142857142857142, 579: 0.6428571428571429, 5500: 0.14285714285714285, 2542: 0.14285714285714285}
{11281: 0.07142857142857142, 579: 0.6428571428571429, 5500: 0.14285714285714285, 2542: 0.14285714285714285}

2) NO RELEVANT CONTEXT; SHOULD RETURN AVERAGES
{11281: 0.25, 579: 0.25, 5500: 0.25, 2542: 0.25}
{11281: 0.25, 579: 0.25, 5500: 0.25, 2542: 0.25}

3) WITH RELEVANT CONTEXT:
{1: 1.0}
{1: 1.0}

4) WITH RELEVANT CONTEXT:
{13776: 0.125, 14803: 0.125, 3341: 0.75}
{13776: 0.125, 14803: 0.125, 3341: 0.75}

5) NO RELEVANT LEMMA; SHOULD RETURN BACKUP (ALL) EQUAL WEIGHT
6) WITH RELEVANT CONTEXT:
{1489: 0.8, 6197: 0.2}
{1489: 0.8, 6197: 0.2}


### ITERATE THROUGH XML ROOT DATA (CAN BE TRAIN, TEST, EVAL) AND RETURN CONTEXTS FOR EACH INSTANCE ID

In [448]:
"""MODIFIED 8-20-17"""

### CREATE CONTEXT DICT BY INSTANCE NUMBER AS ITERATE THROUGH A DOCUMENT

def create_ngram_contexts(root, pos_grams="N") :
    #creates unigrams in format instance to (P_0, lemma/POS)
    #creates bigrams in format instance to (P_1, P_0, lemma/POS)
    #creates trigrams in format instance to (P_2, P_1, P_0, lemma/POS)
    #where:
    #P_2 P_1 P_0 can be either:
    # lemmas --> pos_grams="Y"
    # POS --> pos_grams="N"
    
    #returns unigram bigrams and trigram contexts
    
    """NOTE: Use an indexed keys map to return word sense indexes instead of word senses"""
    
    ### MODIFY TO and "Y" add reverse index to get word sense indexes instead
    """NOTE: n-grams are different than in lectures because WANT to use current context like POS and lemma;
    not just preceding context. This is because we are trying to predict the current WORD SENSE, and already
    have the current POS and lemma (due to Stanford NLP prediction used)"""
    unigram_contexts = defaultdict(dict)
    bigram_contexts = defaultdict(dict)
    trigram_contexts = defaultdict(dict)

    #iterate through docs
    for i in root.getchildren():
#     for i in root.getchildren()[0:2]:
        
        #iterate through sentences
        #if at start of data, no p1/p2
        p_1 = None
        p_2 = None
        instance_id = None
        for j in i.getchildren():
            #iterate through tags
            for k in j.getchildren():
                if pos_grams == "Y" :
                    #use POS for bi/tri grams
                    p_0 = k.attrib.get('pos')
                else :
                    #use lemma for bi/tri-grams
                    p_0 = k.attrib.get('lemma')
                if k.tag == 'instance':                
                    instance_id = k.attrib.get('id')
                    if pos_grams == "Y" :
                        lemma = k.attrib.get('lemma')
                        #NOTE: Need to add lemmas here because need to limit to POS in just that lemma
                        unigram_contexts[instance_id] = (p_0,lemma)
                        bigram_contexts[instance_id] = (p_1,p_0,lemma)
                        trigram_contexts[instance_id] = (p_2,p_1,p_0,lemma)
                        
                    else :
                        unigram_contexts[instance_id] = (p_0)
                        bigram_contexts[instance_id] = (p_1,p_0)
                        trigram_contexts[instance_id] = (p_2,p_1,p_0)
                #set lemmas = prior lemmas
                p_2 = p_1
                p_1 = p_0
    
    return unigram_contexts, bigram_contexts, trigram_contexts


### TRAIN VERSION ###
#NOTE: using indexed version of maps
train_context_unigrams, train_context_bigrams, train_context_trigrams = create_ngram_contexts(
    semcortrain_root, pos_grams="N")

train_context_unigrams_POS, train_context_bigrams_POS, train_context_trigrams_POS = create_ngram_contexts(
    semcortrain_root, pos_grams="Y")


"""8_20_17_EVAL"""
#--> Change semcortestroot references to Eval

### TEST VERSION ###
#NOTE: using indexed version of maps
test_context_unigrams, test_context_bigrams, test_context_trigrams = create_ngram_contexts(
    var_test_root, pos_grams="N")

test_context_unigrams_POS, test_context_bigrams_POS, test_context_trigrams_POS = create_ngram_contexts(
    var_test_root, pos_grams="Y")


print "UNIGRAMS (lemmas):", list(itertools.islice(train_context_unigrams.iteritems(), 0, 5))
print
print "BIGRAMS (lemmas):", list(itertools.islice(train_context_bigrams.iteritems(), 0, 5))
print
print "TRIGRAMS (lemmas):", list(itertools.islice(train_context_trigrams.iteritems(), 0, 5))
print 
print "UNIGRAMS (POS|curr lemma):", list(itertools.islice(train_context_unigrams_POS.iteritems(), 0, 5))
print
print "BIGRAMS (POS|curr lemma):", list(itertools.islice(train_context_bigrams_POS.iteritems(), 0, 5))
print
print "TRIGRAMS (POS|curr lemma):", list(itertools.islice(train_context_trigrams_POS.iteritems(), 0, 5))

/home/cscaudill/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:27: DeprecationWarning:

This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.

/home/cscaudill/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:35: DeprecationWarning:

This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.

/home/cscaudill/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:37: DeprecationWarning:

This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.



UNIGRAMS (lemmas): [('d032.s132.t004', 'iron'), ('d032.s132.t005', 'tire'), ('d032.s132.t006', 'grip'), ('d032.s132.t007', 'wheel'), ('d032.s132.t000', 'spoke')]

BIGRAMS (lemmas): [('d032.s132.t004', ('the', 'iron')), ('d032.s132.t005', ('iron', 'tire')), ('d032.s132.t006', ('tire', 'grip')), ('d032.s132.t007', ('the', 'wheel')), ('d032.s132.t000', ('the', 'spoke'))]

TRIGRAMS (lemmas): [('d032.s132.t004', (',', 'the', 'iron')), ('d032.s132.t005', ('the', 'iron', 'tire')), ('d032.s132.t006', ('iron', 'tire', 'grip')), ('d032.s132.t007', ('onto', 'the', 'wheel')), ('d032.s132.t000', ('.', 'the', 'spoke'))]

UNIGRAMS (POS|curr lemma): [('d032.s132.t004', ('NOUN', 'iron')), ('d032.s132.t005', ('NOUN', 'tire')), ('d032.s132.t006', ('VERB', 'grip')), ('d032.s132.t007', ('NOUN', 'wheel')), ('d032.s132.t000', ('NOUN', 'spoke'))]

BIGRAMS (POS|curr lemma): [('d032.s132.t004', ('DET', 'NOUN', 'iron')), ('d032.s132.t005', ('NOUN', 'NOUN', 'tire')), ('d032.s132.t006', ('NOUN', 'VERB', 'grip')), 

### CREATE DENSE DICT OF MODEL PREDICTIONS
 - Note: Be careful with all variables to ensure correct info gets fed to each model

In [449]:
def create_predictions_dict(context_dict, 
                             n, 
                             normalized_ngrams, 
                             normalized_unigrams_equal_prob,
                             backup_index,
                             POS='N') :
    
    predictions_dict = defaultdict(dict)
    counter = 0
    for instance_id, context in context_dict.iteritems() :
        if type(context) is str :
            #for context of only single lemma; doesn't come in tuple (comes in string); last entry would return single letter
            lemma = context
        else :
            #when comes in as a tuple, can take the last entry as the lemma
            lemma = context[-1]
        predictions = ws_probas(lemma, context, n, normalized_ngrams, normalized_unigrams_equal_prob, backup_index, POS)
        predictions_dict[instance_id] = predictions
        
    return predictions_dict

"""
### WILL DIFFER:
 - context_dict (can be train, test, POS, n_gram, etc)
 - n
 - normalized_ngrams (train ONLY, can be POS, n_gram, etc)
 - POS
 
### WILL BE CONSISTENT: 
 - normalized_unigrams_equal_prob
 - backup index
"""

### TRAIN ###
unigram_pred_train_dense = create_predictions_dict(context_dict=train_context_unigrams,
                                                   n=1,
                                                   normalized_ngrams=normalized_unigrams,
                                                   normalized_unigrams_equal_prob=normalized_unigrams_equal_prob,
                                                   backup_index = backup_index_probs,
                                                   POS='N')
bigram_pred_train_dense = create_predictions_dict(context_dict=train_context_bigrams,
                                                   n=2,
                                                   normalized_ngrams=normalized_bigrams,
                                                   normalized_unigrams_equal_prob=normalized_unigrams_equal_prob,
                                                   backup_index = backup_index_probs,
                                                   POS='N')
trigram_pred_train_dense = create_predictions_dict(context_dict=train_context_trigrams,
                                                   n=3,
                                                   normalized_ngrams=normalized_trigrams,
                                                   normalized_unigrams_equal_prob=normalized_unigrams_equal_prob,
                                                   backup_index = backup_index_probs,
                                                   POS='N')
unigram_POS_pred_train_dense = create_predictions_dict(context_dict=train_context_unigrams_POS,
                                                   n=1,
                                                   normalized_ngrams=normalized_unigrams_POS,
                                                   normalized_unigrams_equal_prob=normalized_unigrams_equal_prob,
                                                   backup_index = backup_index_probs,
                                                   POS='N')
bigram_POS_pred_train_dense = create_predictions_dict(context_dict=train_context_bigrams_POS,
                                                   n=2,
                                                   normalized_ngrams=normalized_bigrams_POS,
                                                   normalized_unigrams_equal_prob=normalized_unigrams_equal_prob,
                                                   backup_index = backup_index_probs,
                                                   POS='N')
trigram_POS_pred_train_dense = create_predictions_dict(context_dict=train_context_trigrams_POS,
                                                   n=3,
                                                   normalized_ngrams=normalized_trigrams_POS,
                                                   normalized_unigrams_equal_prob=normalized_unigrams_equal_prob,
                                                   backup_index = backup_index_probs,
                                                   POS='N')

### TEST ###
unigram_pred_test_dense = create_predictions_dict(context_dict=test_context_unigrams,
                                                   n=1,
                                                   normalized_ngrams=normalized_unigrams,
                                                   normalized_unigrams_equal_prob=normalized_unigrams_equal_prob,
                                                   backup_index = backup_index_probs,
                                                   POS='N')
bigram_pred_test_dense = create_predictions_dict(context_dict=test_context_bigrams,
                                                   n=2,
                                                   normalized_ngrams=normalized_bigrams,
                                                   normalized_unigrams_equal_prob=normalized_unigrams_equal_prob,
                                                   backup_index = backup_index_probs,
                                                   POS='N')
trigram_pred_test_dense = create_predictions_dict(context_dict=test_context_trigrams,
                                                   n=3,
                                                   normalized_ngrams=normalized_trigrams,
                                                   normalized_unigrams_equal_prob=normalized_unigrams_equal_prob,
                                                   backup_index = backup_index_probs,
                                                   POS='N')
unigram_POS_pred_test_dense = create_predictions_dict(context_dict=test_context_unigrams_POS,
                                                   n=1,
                                                   normalized_ngrams=normalized_unigrams_POS,
                                                   normalized_unigrams_equal_prob=normalized_unigrams_equal_prob,
                                                   backup_index = backup_index_probs,
                                                   POS='N')
bigram_POS_pred_test_dense = create_predictions_dict(context_dict=test_context_bigrams_POS,
                                                   n=2,
                                                   normalized_ngrams=normalized_bigrams_POS,
                                                   normalized_unigrams_equal_prob=normalized_unigrams_equal_prob,
                                                   backup_index = backup_index_probs,
                                                   POS='N')
trigram_POS_pred_test_dense = create_predictions_dict(context_dict=test_context_trigrams_POS,
                                                   n=3,
                                                   normalized_ngrams=normalized_trigrams_POS,
                                                   normalized_unigrams_equal_prob=normalized_unigrams_equal_prob,
                                                   backup_index = backup_index_probs,
                                                   POS='N')



print "UNIGRAM PRED (MFS):", list(itertools.islice(unigram_pred_train_dense.iteritems(), 0, 3))
print
print "BIGRAM PRED:", list(itertools.islice(bigram_pred_train_dense.iteritems(), 0, 3))
print    
print "TRIGRAM PRED:", list(itertools.islice(trigram_pred_train_dense.iteritems(), 0, 3))
print    
print "UNIGRAM PRED (POS):", list(itertools.islice(unigram_POS_pred_train_dense.iteritems(), 0, 3))
print
print "BIGRAM PRED (POS):", list(itertools.islice(bigram_POS_pred_train_dense.iteritems(), 0, 3))
print    
print "TRIGRAM PRED (POS):", list(itertools.islice(trigram_POS_pred_train_dense.iteritems(), 0, 3))
print  

print unigram_pred_test_dense['d000.s000.t000']


UNIGRAM PRED (MFS): [('d032.s132.t004', {13776: 0.125, 14803: 0.125, 3341: 0.75}), ('d032.s132.t005', {11920: 0.21428571428571427, 4825: 0.5714285714285714, 10970: 0.21428571428571427}), ('d032.s132.t006', {4452: 0.4090909090909091, 5255: 0.3181818181818182, 18312: 0.09090909090909091, 31020: 0.045454545454545456, 27920: 0.045454545454545456, 15956: 0.09090909090909091})]

BIGRAM PRED: [('d032.s132.t004', {3341: 1.0}), ('d032.s132.t005', {4825: 1.0}), ('d032.s132.t006', {4452: 1.0})]

TRIGRAM PRED: [('d032.s132.t004', {3341: 1.0}), ('d032.s132.t005', {4825: 1.0}), ('d032.s132.t006', {4452: 1.0})]

UNIGRAM PRED (POS): [('d032.s132.t004', {14803: 0.14285714285714285, 3341: 0.8571428571428571}), ('d032.s132.t005', {4825: 1.0}), ('d032.s132.t006', {18312: 0.15384615384615385, 4452: 0.6923076923076923, 15956: 0.15384615384615385})]

BIGRAM PRED (POS): [('d032.s132.t004', {3341: 1.0}), ('d032.s132.t005', {4825: 1.0}), ('d032.s132.t006', {4452: 0.8, 15956: 0.2})]

TRIGRAM PRED (POS): [('d032.

# RUN INDIVIDUAL MODELS AND GET STATS

### CREATE MODEL STATS PRINTER

In [450]:
import operator

def print_model_stats(pred_dense, ids_array, max_samples = None, voting_classifier=None) :
    #function to iterate through to see how many times the highest prob prediction is correct
    #also gives prop
    
    if max_samples is not None :
        num_samples = max_samples
    else :
        num_samples = len(ids_array)

        
    counter = 0
    correct_pred = 0                      #binary 1/0 credit given
    correct_pred_weight = 0              #if max prob predicts correctly, give max prob
    weight_correct_sense = 0             #prob assigned to correct sense, whether max or not


    for (instance_id, preds), act_index in zip(sorted(pred_dense.iteritems()), ids_array) :
        #assign relevant scoring if correct or if correct word sense within predictions
        #note: pred_dense must be sorted to ensure comes in same order as ids_array
        if act_index == 0 or preds is None:
            #if <unk> (index=0), give no credit. Saves time because does not iterate through a full/large dense array of predictions.
            #preds comes in None from voting classifier when encountering act_index==0
            correct_pred += 0
            correct_pred_weight += 0
        else :
            #get maximum probability (sorted makes default to first index if there is a tie)
            max_index = max(sorted(preds.iteritems()), key=operator.itemgetter(1))[0]
            max_prob = max(sorted(preds.iteritems()), key=operator.itemgetter(1))[1]
            if max_index == act_index :
                #correct maximum
                correct_pred += 1
                correct_pred_weight += max_prob
                weight_correct_sense += preds[act_index]
            elif act_index in preds.keys() :
                #not correct but probability was assigned
                weight_correct_sense += preds[act_index]
                
        counter += 1
        if counter >= max_samples and max_samples is not None:
            break

    correct_pred_rate = float(correct_pred)/num_samples
    correct_pred_weight_rate = float(correct_pred_weight)/num_samples
    weight_correct_sense_rate = float(weight_correct_sense)/num_samples
        
    print "NUM SAMPLES:\t\t\t", num_samples
    print "CORRECT PREDS:\t\t\t", correct_pred, "/   ", '{:.1%}'.format(correct_pred_rate)
    print "CORRECT PRED WEIGHT ASSIGNED:\t", "{:0.1f}".format(correct_pred_weight), "/   ", '{:.1%}'.format(correct_pred_weight_rate) 
    print "TOTAL WEIGHT TO CORRECT SENSE:\t", "{:0.1f}".format(weight_correct_sense), "/   ", '{:.1%}'.format(weight_correct_sense_rate)

print "UNIGRAM LEMMA (MFS MODEL): TEST DATA"
print_model_stats(unigram_pred_test_dense, var_test_ids, max_samples = None)
print
print "BIGRAM LEMMA: TEST DATA"
print_model_stats(bigram_pred_test_dense, var_test_ids, max_samples = None)
print 
print "TRIGRAM LEMMA: TEST DATA"
print_model_stats(trigram_pred_test_dense, var_test_ids, max_samples = None)
print
print "UNIGRAM POS: TEST DATA"
print_model_stats(unigram_POS_pred_test_dense, var_test_ids, max_samples = None)
print
print "BIGRAM POS: TEST DATA"
print_model_stats(bigram_POS_pred_test_dense, var_test_ids, max_samples = None)
print 
print "TRIGRAM POS: TEST DATA"
print_model_stats(trigram_POS_pred_test_dense, var_test_ids, max_samples = None)
print
#edited 8_19_17_KO
print "LSTM STATS: TEST DATA"
print_model_stats(lstm_dense_preds_test_matched_index, var_test_ids, max_samples = None)
print


UNIGRAM LEMMA (MFS MODEL): TEST DATA
NUM SAMPLES:			1850
CORRECT PREDS:			1008 /    54.5%
CORRECT PRED WEIGHT ASSIGNED:	778.6 /    42.1%
TOTAL WEIGHT TO CORRECT SENSE:	894.4 /    48.3%

BIGRAM LEMMA: TEST DATA
NUM SAMPLES:			1850
CORRECT PREDS:			1041 /    56.3%
CORRECT PRED WEIGHT ASSIGNED:	751.6 /    40.6%
TOTAL WEIGHT TO CORRECT SENSE:	854.9 /    46.2%

TRIGRAM LEMMA: TEST DATA
NUM SAMPLES:			1850
CORRECT PREDS:			1015 /    54.9%
CORRECT PRED WEIGHT ASSIGNED:	598.4 /    32.3%
TOTAL WEIGHT TO CORRECT SENSE:	715.3 /    38.7%

UNIGRAM POS: TEST DATA
NUM SAMPLES:			1850
CORRECT PREDS:			1136 /    61.4%
CORRECT PRED WEIGHT ASSIGNED:	913.5 /    49.4%
TOTAL WEIGHT TO CORRECT SENSE:	1006.9 /    54.4%

BIGRAM POS: TEST DATA
NUM SAMPLES:			1850
CORRECT PREDS:			1119 /    60.5%
CORRECT PRED WEIGHT ASSIGNED:	928.1 /    50.2%
TOTAL WEIGHT TO CORRECT SENSE:	1013.8 /    54.8%

TRIGRAM POS: TEST DATA
NUM SAMPLES:			1850
CORRECT PREDS:			1057 /    57.1%
CORRECT PRED WEIGHT ASSIGNED:	840.2 /    45.4%

### CREATE MODEL STATS RETURNER (SAME EXACT FUNCTION AS PRINT_MODEL_STATS)

In [451]:
import operator

def return_model_stats(pred_dense, ids_array, max_samples = None, voting_classifier=None) :
    #function to iterate through to see how many times the highest prob prediction is correct
    #also gives prop
    
    if max_samples is not None :
        num_samples = max_samples
    else :
        num_samples = len(ids_array)

        
    counter = 0
    correct_pred = 0                      #binary 1/0 credit given
    correct_pred_weight = 0              #if max prob predicts correctly, give max prob
    weight_correct_sense = 0             #prob assigned to correct sense, whether max or not


    for (instance_id, preds), act_index in zip(sorted(pred_dense.iteritems()), ids_array) :
        #assign relevant scoring if correct or if correct word sense within predictions
        #note: pred_dense must be sorted to ensure comes in same order as ids_array
        if act_index == 0 or preds is None:
            #if <unk> (index=0), give no credit. Saves time because does not iterate through a full/large dense array of predictions.
            #preds comes in None from voting classifier when encountering act_index==0
            correct_pred += 0
            correct_pred_weight += 0
        else :
            #get maximum probability (sorted makes default to first index if there is a tie)
            max_index = max(sorted(preds.iteritems()), key=operator.itemgetter(1))[0]
            max_prob = max(sorted(preds.iteritems()), key=operator.itemgetter(1))[1]
            if max_index == act_index :
                #correct maximum
                correct_pred += 1
                correct_pred_weight += max_prob
                weight_correct_sense += preds[act_index]
            elif act_index in preds.keys() :
                #not correct but probability was assigned
                weight_correct_sense += preds[act_index]
                
        counter += 1
        if counter >= max_samples and max_samples is not None:
            break

    correct_pred_rate = float(correct_pred)/num_samples
    correct_pred_weight_rate = float(correct_pred_weight)/num_samples
    weight_correct_sense_rate = float(weight_correct_sense)/num_samples
    
    return num_samples, correct_pred, correct_pred_weight, weight_correct_sense

### VOTING CLASSIFIER FUNCTION (BOTH HARD AND SOFT CLASSIFIERS RETURNED)

In [452]:
def voting_classifier_dense_preds(pred_dense_model_list, ids_array, weights_list=None, max_samples=None) :

    """
    Takes in:
    pred_dense_model_list: model predictions in dense format
    ids_array: corresponding train or test id actual word senses (by index)
    weights_list (optional): Optional weights list for re-weighting in SOFT CLASSIFIER ONLY. None returns even weighting.
    max_samples (optional): Stop each model after a certain amount of instance_ids. None returns all instance_ids
    
    RETURNS
    vote_counts -> Dense dictionary by instance_id adding 1/num_models to each max sense predicted per model
    weight_counts -> Dense dictionary by instance_id adding weight to any sense predicted per model
    
    NOTE: returns in same dense dictionary format that we can run through print_model_stats() for evaluation
    vote_counts -> Used for HARD VOTING CLASSIFIER EVAL
    
    """

    num_models = len(pred_dense_model_list)
    
    if weights_list is None :
        #defaults to even weighting
        weights_list = [1/float(num_models) for m in pred_dense_model_list]
    elif sum(weights_list) > 1.01 or sum(weights_list) < .99:
        print "RE-DO: WEIGHTS DO NOT SUM TO 1.00"
    
    if max_samples is not None :
        num_samples = max_samples
    else :
        num_samples = len(ids_array)
    
    
    vote_counts = defaultdict(lambda: defaultdict(lambda: 0.0))
    weight_counts = defaultdict(lambda: defaultdict(lambda: 0.0))
    
    model_counter = 0    
    for model_preds in pred_dense_model_list :
        #get key info from each predictive model output, append to a default_dict to be referenced later
        
        counter = 0
        for (instance_id, preds), act_index in zip(sorted(model_preds.iteritems()), ids_array) :            
            
            #KO added 8_19_17_KO
            if preds is None :
                #handle None from LSTM
                continue
            
            #assign relevant scoring if correct or if correct word sense within predictions
            #note: pred_dense must be sorted to ensure comes in same order as ids_array
            if act_index == 0 :
                #if unk, do nothing. No model will get correct.
                #assigning to none will allow to filter out when printing stats
                vote_counts[instance_id] = None
                
                weight_counts[instance_id][act_index] = None                
                #ORIG
#                 weight_counts[instance_id][index] = None
            
            else :
                #get maximum probability (sorted makes default to first index if there is a tie)
                max_index = max(sorted(preds.iteritems()), key=operator.itemgetter(1))[0]
                max_prob = max(sorted(preds.iteritems()), key=operator.itemgetter(1))[1]
                vote_counts[instance_id][max_index] += 1/float(num_models)
                #assign a weighted amount for each prediction within model for instance id
                for index, prob in preds.iteritems() :
                    weight_counts[instance_id][index] += prob*weights_list[model_counter] #picks up corresponding weight for model
                            
            counter += 1
            if counter >= max_samples and max_samples is not None:
                break
                
        model_counter += 1
    
    return vote_counts, weight_counts

model_list = [unigram_pred_test_dense,unigram_POS_pred_test_dense]
model_weights = [0.4, 0.6]
max_samples = None

hard, soft = voting_classifier_dense_preds(pred_dense_model_list=model_list, ids_array=var_test_ids,
                                     weights_list=model_weights, max_samples=max_samples)

#KO MODIFIED TO 50 SAMPLES
print "HARD VOTING CLASSIFIER STATS:"
print_model_stats(hard, var_test_ids, max_samples=None)
print 
print "SOFT VOTING CLASSIFIER STATS:"
print_model_stats(soft, var_test_ids, max_samples=None)

HARD VOTING CLASSIFIER STATS:
NUM SAMPLES:			1850
CORRECT PREDS:			1008 /    54.5%
CORRECT PRED WEIGHT ASSIGNED:	1008.0 /    54.5%
TOTAL WEIGHT TO CORRECT SENSE:	1072.0 /    57.9%

SOFT VOTING CLASSIFIER STATS:
NUM SAMPLES:			1850
CORRECT PREDS:			1136 /    61.4%
CORRECT PRED WEIGHT ASSIGNED:	872.1 /    47.1%
TOTAL WEIGHT TO CORRECT SENSE:	961.9 /    52.0%


### IMPLEMENT GRID SEARCH FOR OPTIMAL ENSEMBLES (EVEN WEIGHTING)
NOTE: Only use to find optimal models in test set; not in Eval set. Must reuse optimal models from test set.

In [471]:
"""CURRENTLY USING TEST_IDS, MAX_SAMPLES=NONE, ETC... WILL WANT TO STANDARDIZE INTO A FUNCTION"""
"""ONCE IN FUNCTION, ADD AN OPTIMAL WEIGHTS FUNCTION """

def grid_search_ensembles(model_map, model_name_map, correct_labels, max_samples=None) :
    
    """FUNCTION RETURNING KEY STATISTICS FOR EVERY MODEL COMBINATION AFTER RUNNING EACH MODEL ENSEMBLE
    THROUGH HARD AND SOFT VOTING CLASSIFIERS    
    """

    #will add combo numbers to a combos_list
    combos_list = []

    #get combo numbers
    for num in xrange(1, len(model_map.keys())):
        #note: not 0 because 0 returns combo of length zero, which we don't want
        for combo in itertools.combinations(model_map.keys(), num+1):
            combos_list.append(combo)
    
    #to be used to append names and stats to
    ensemble_tracker = []


    #iterate through each set of combo numbers corresponding to a model
    for model_nums in combos_list:
        #get models and names associated
        model_list = [model_map[num] for num in model_nums]
        model_names_list = [model_name_map[num] for num in model_nums]

        #load into hard and soft classifier
        hard, soft = voting_classifier_dense_preds(pred_dense_model_list=model_list, ids_array=correct_labels,
                                         weights_list=None, max_samples=max_samples)

        #return hard classifier output, add to ensemble_tracker
        num_samples, correct_pred, correct_pred_weight, weight_correct_sense = return_model_stats(
            hard, correct_labels, max_samples=max_samples)

        correct_pred_rate = float(correct_pred)/num_samples
        correct_pred_weight_rate = float(correct_pred_weight)/num_samples
        weight_correct_sense_rate = float(weight_correct_sense)/num_samples


        ensemble_tracker.append((model_names_list, model_nums, "HARD", 
                                num_samples, correct_pred, correct_pred_weight, weight_correct_sense,
                               correct_pred_rate,correct_pred_weight_rate,weight_correct_sense_rate))


        #return soft classifier output, add to ensemble_tracker
        num_samples, correct_pred, correct_pred_weight, weight_correct_sense = return_model_stats(
            soft, correct_labels, max_samples=max_samples)

        correct_pred_rate = float(correct_pred)/num_samples
        correct_pred_weight_rate = float(correct_pred_weight)/num_samples
        weight_correct_sense_rate = float(weight_correct_sense)/num_samples


        ensemble_tracker.append((model_names_list, model_nums, "SOFT", 
                                num_samples, correct_pred, correct_pred_weight, weight_correct_sense,
                               correct_pred_rate,correct_pred_weight_rate,weight_correct_sense_rate))
        
    return ensemble_tracker



###RETURN ENSEMBLE STATS DATA###

#set up models want to iterate through and get all combos of models for
model_map = {0: unigram_pred_test_dense,1:bigram_pred_test_dense,2:trigram_pred_test_dense,
          3:unigram_POS_pred_test_dense,4:bigram_POS_pred_test_dense,5:trigram_POS_pred_test_dense,
             6:lstm_dense_preds_test_matched_index}

#set up model names (used for prints)
model_name_map = {0: 'unigram_pred_test_dense',1:'bigram_pred_test_dense',2:'trigram_pred_test_dense',
          3:'unigram_POS_pred_test_dense',4:'bigram_POS_pred_test_dense',5:'trigram_POS_pred_test_dense',
                 6: 'lstm_dense_preds_test_matched_index'}


ensemble_data = grid_search_ensembles(model_map=model_map,model_name_map=model_name_map,
                                         correct_labels=var_test_ids, max_samples=None)


KeyboardInterrupt: 

### FUNCTIONS TO RETURN AND PRINT BEST ENSEMBLE INFO

In [454]:
def return_top_ensemble_info(ensemble_data, n=5) :
    #RETURN LIST OF TOP ENSEMBLE INFO (use if need to reference model numbers, model names directly)
    
    #sort by number correct
    ensemble_data.sort(key=lambda x: x[4], reverse=True)
    
    #get top n models
    ensemble_data = ensemble_data[0:n]
    
    return ensemble_data


def print_top_ensemble_info(ensemble_data, n=5) :

    print "TOP", str(n),"MODELS:"
    print "\n"

    #sort by number correct
    ensemble_data.sort(key=lambda x: x[4], reverse=True)
    
    #get top n models
    ensemble_data = ensemble_data[0:n]
    
    for item in ensemble_data[0:n]:
        model_names_list = item[0]
        model_nums = item[1]
        voting_classifier = item[2]
        num_samples = item[3]
        correct_pred = item[4]
        correct_pred_weight = item[5]
        weight_correct_sense = item[6]
        correct_pred_rate = item[7]
        correct_pred_weight_rate = item[8]
        weight_correct_sense_rate = item[9]


        print "COMBINED MODELS:", model_names_list
        print "MODEL NUMBERS:", model_nums
        print "VOTING CLASSIFIER TYPE:", voting_classifier
        print "NUM SAMPLES:", num_samples
        print "CORRECT PREDS:\t\t\t", correct_pred, "/   ", '{:.1%}'.format(correct_pred_rate)
        print "CORRECT PRED WEIGHT ASSIGNED:\t", "{:0.0f}".format(correct_pred_weight), "/   ", '{:.1%}'.format(correct_pred_weight_rate) 
        print "TOTAL WEIGHT TO CORRECT SENSE:\t", "{:0.0f}".format(weight_correct_sense), "/   ", '{:.1%}'.format(weight_correct_sense_rate)
        print 


#use if want to reference model numbers, model names, scores directly
top_ensemble_info = return_top_ensemble_info(ensemble_data=ensemble_data, n=5)

print_top_ensemble_info(ensemble_data=ensemble_data, n=5)

TOP 5 MODELS:


COMBINED MODELS: ['trigram_pred_test_dense', 'unigram_POS_pred_test_dense', 'bigram_POS_pred_test_dense']
MODEL NUMBERS: (2, 3, 4)
VOTING CLASSIFIER TYPE: SOFT
NUM SAMPLES: 1850
CORRECT PREDS:			1156 /    62.5%
CORRECT PRED WEIGHT ASSIGNED:	821 /    44.4%
TOTAL WEIGHT TO CORRECT SENSE:	912 /    49.3%

COMBINED MODELS: ['unigram_POS_pred_test_dense', 'bigram_POS_pred_test_dense']
MODEL NUMBERS: (3, 4)
VOTING CLASSIFIER TYPE: SOFT
NUM SAMPLES: 1850
CORRECT PREDS:			1152 /    62.3%
CORRECT PRED WEIGHT ASSIGNED:	921 /    49.8%
TOTAL WEIGHT TO CORRECT SENSE:	1010 /    54.6%

COMBINED MODELS: ['bigram_pred_test_dense', 'trigram_pred_test_dense', 'unigram_POS_pred_test_dense', 'bigram_POS_pred_test_dense']
MODEL NUMBERS: (1, 2, 3, 4)
VOTING CLASSIFIER TYPE: SOFT
NUM SAMPLES: 1850
CORRECT PREDS:			1150 /    62.2%
CORRECT PRED WEIGHT ASSIGNED:	807 /    43.6%
TOTAL WEIGHT TO CORRECT SENSE:	898 /    48.5%

COMBINED MODELS: ['unigram_pred_test_dense', 'bigram_pred_test_dense', 'tri

### FUNCTIONS TO COMPARE MODEL ENSEMBLES WITH AND WITHOUT SINGLE MODEL
Note: Only to be used when running on test set

In [455]:
def print_ensemble_target_model_info(ensemble_data,var_test_ids, model_name_map, model_map, target_model) :
    #sort by number correct
    ensemble_data.sort(key=lambda x: x[4], reverse=True)
    
    #get target models
    target_model_ensembles = []
    target_model_ensembles_num = 0
    target_model_ensembles_rate = 0
    #get exact same ensembles, just without target
    same_ensembles_without_target = []
    same_ensembles_without_target_num = 0
    same_ensembles_without_target_rate = 0
    
    #get models lists
    for item in ensemble_data:         
        model_nums = item[1]
        #models with target model in and at least three models
        if target_model in model_nums and len(model_nums) > 1:
            target_model_ensembles.append(model_nums)
    for models in target_model_ensembles :
        #remove target
        models_list = []
        for model in models :
            if model <> target_model :
                models_list.append(model)
        models_list = tuple(models_list)
        same_ensembles_without_target.append(models_list)
    
    
    for item in ensemble_data:
        model_names_list = item[0]
        model_nums = item[1]
        voting_classifier = item[2]
        num_samples = item[3]
        correct_pred = item[4]
        correct_pred_weight = item[5]
        weight_correct_sense = item[6]
        correct_pred_rate = item[7]
        correct_pred_weight_rate = item[8]
        weight_correct_sense_rate = item[9]
        
        
        if model_nums in target_model_ensembles:
            target_model_ensembles_num += 1
            target_model_ensembles_rate += correct_pred_rate
            
        if model_nums in same_ensembles_without_target:
            same_ensembles_without_target_num += 1
            same_ensembles_without_target_rate += correct_pred_rate
        
        #get single model
        if len(model_nums) == 2 and model_nums in target_model_ensembles:
            #remove target
            single_model = None
            for model in model_nums :
                if model <> target_model :
                    single_model = model
                    samples,correct,_,_ = return_model_stats(
                      model_map[single_model], var_test_ids, max_samples = None)
            same_ensembles_without_target_num +=1 
            same_ensembles_without_target_rate += float(correct)/float(samples)
    
    #normalize
    target_model_ensembles_rate = float(target_model_ensembles_rate)/target_model_ensembles_num
    same_ensembles_without_target_rate = float(same_ensembles_without_target_rate)/same_ensembles_without_target_num
    target_model_contribution = target_model_ensembles_rate - same_ensembles_without_target_rate
        
    print "WITH TARGET MODEL #", target_model, ":"
    print "MODEL:", model_name_map[target_model]
    print "NUM ENSEMBLES:", target_model_ensembles_num, same_ensembles_without_target_num
    print "ACCURACY WITH:", '{:.1%}'.format(target_model_ensembles_rate)
    print "ACCURACY WITHOUT:", '{:.1%}'.format(same_ensembles_without_target_rate)    
    print "AVERAGE CONTRIBUTION:", '{:.1%}'.format(target_model_contribution)
    print 
        
print_ensemble_target_model_info(ensemble_data=ensemble_data, var_test_ids=var_test_ids, model_map=model_map, model_name_map=model_name_map, target_model=0)
print_ensemble_target_model_info(ensemble_data=ensemble_data, var_test_ids=var_test_ids,model_map=model_map,model_name_map=model_name_map,target_model=1)
print_ensemble_target_model_info(ensemble_data=ensemble_data, var_test_ids=var_test_ids,model_map=model_map,model_name_map=model_name_map,target_model=2)
print_ensemble_target_model_info(ensemble_data=ensemble_data, var_test_ids=var_test_ids,model_map=model_map,model_name_map=model_name_map,target_model=3)
print_ensemble_target_model_info(ensemble_data=ensemble_data, var_test_ids=var_test_ids,model_map=model_map,model_name_map=model_name_map,target_model=4)
print_ensemble_target_model_info(ensemble_data=ensemble_data, var_test_ids=var_test_ids,model_map=model_map,model_name_map=model_name_map,target_model=5)
print_ensemble_target_model_info(ensemble_data=ensemble_data, var_test_ids=var_test_ids,model_map=model_map,model_name_map=model_name_map,target_model=6)

WITH TARGET MODEL # 0 :
MODEL: unigram_pred_test_dense
NUM ENSEMBLES: 126 126
ACCURACY WITH: 57.5%
ACCURACY WITHOUT: 57.6%
AVERAGE CONTRIBUTION: -0.0%

WITH TARGET MODEL # 1 :
MODEL: bigram_pred_test_dense
NUM ENSEMBLES: 126 126
ACCURACY WITH: 57.7%
ACCURACY WITHOUT: 57.3%
AVERAGE CONTRIBUTION: 0.4%

WITH TARGET MODEL # 2 :
MODEL: trigram_pred_test_dense
NUM ENSEMBLES: 126 126
ACCURACY WITH: 57.5%
ACCURACY WITHOUT: 57.6%
AVERAGE CONTRIBUTION: -0.1%

WITH TARGET MODEL # 3 :
MODEL: unigram_POS_pred_test_dense
NUM ENSEMBLES: 126 126
ACCURACY WITH: 58.8%
ACCURACY WITHOUT: 56.2%
AVERAGE CONTRIBUTION: 2.6%

WITH TARGET MODEL # 4 :
MODEL: bigram_POS_pred_test_dense
NUM ENSEMBLES: 126 126
ACCURACY WITH: 58.7%
ACCURACY WITHOUT: 56.2%
AVERAGE CONTRIBUTION: 2.5%

WITH TARGET MODEL # 5 :
MODEL: trigram_POS_pred_test_dense
NUM ENSEMBLES: 126 126
ACCURACY WITH: 58.1%
ACCURACY WITHOUT: 56.9%
AVERAGE CONTRIBUTION: 1.2%

WITH TARGET MODEL # 6 :
MODEL: lstm_dense_preds_test_matched_index
NUM ENSEMBLES: 

In [456]:
#INTENTIONALLY CREATE ERROR TO MAKE CODE STOP RUNNING
error = {}
error.append('ERROR')

AttributeError: 'dict' object has no attribute 'append'

### IMPLEMENT STOCHASTIC GRADIENT DESCENT TO GET OPTIMAL WEIGHTS FOR AN ENSEMBLE
 - Note: Only built for soft classifier as results clearly show soft classifier is superior
 - Note: Only use to find optimal models in test set; not in Eval set. Must reuse optimal weights from test set.

In [459]:
### SHOULD I IMPLEMENT WEIGHTS RANDOMLY THEN AVERAGE THEM AFTERWARDS?

def get_optimal_weights_soft(model_nums, model_map, model_name_map, correct_labels,weight_step=.01, 
                        epsilon=0.001, num_samples=None, num_epochs=5, batch_size=10,
                             random_init=False, random_sampling=False) :
    
    """IMPLEMENT STOCHASTIC GRADIENT DESCENT TO GET OPTIMAL WEIGHTS (at least locally optimal)
    model_nums = integers corresponding to model index number
    model_map = map containing model dense predictions by model index number
    model_name_map = map containing model name by model index number
    correct_labels = correct word sense IDs (must be ordered)
    weight_step = within stochastic weightings, how much to increase target weight
    epsilon = if target weight resulted in increase to # correct, increase output weights by epsilon
    num_samples = Maximum number of samples (cuts off after)
    num_epochs = number of times to iterate through full prediction set
    batch_size = size of batch to run stochastic gradient descent on
    random_init = initialize weights randomly (True/False); False defaults to even original weights list
    random_sampling = sample batches (True/False); False defaults to iterate through indexes in order
    """
    
    n_models = len(model_nums)
    
    #if model weight is < weight_step (technically, (weight_step/remaining models)), we don't want to decrease it any more or it
    #could go below 0 weight
    models_under_lower_limit = 0
    model_lower_limit = weight_step

    model_upper_limit = 1-weight_step
    
    if num_samples == None :
        num_samples = len(correct_labels)
    else :
        correct_labels = correct_labels[0:num_samples]
    
    if random_init is False :
        #initialize weights evenly
        weights_list = [1/float(n_models) for model in range(n_models)]
    else :
        #initialize randomly and normalize to 1
        weights_list = np.random.random_sample(n_models)
        weights_list /= weights_list.sum()
        weights_list.tolist()
    
    
    #get models, make sure IDs are in sorted order
    model_list = [sorted(model_map[num].iteritems()) for num in model_nums]
    model_names_list = [model_name_map[num] for num in model_nums]
    
    weights_list = list(weights_list)
    
    iteration_counter = 0
    for iteration in range(num_epochs) :
        
        #set initial batch slices
        min_slice = 0
        max_slice = batch_size
                
        #get number of batches
        if num_samples % batch_size == 0 :
            num_batches = num_samples/batch_size
        else :
            num_batches = int(num_samples/batch_size) + 1
        
        for batch in range(num_batches) :

            if random_sampling is True :
                #override min_slice and max_slice to a randomly chosen sample
                min_slice = np.random.randint(0, num_samples)
                max_slice = min_slice+10
            
            #get act_index, model predictions in batch size of instance IDs            
            ids_batch = correct_labels[min_slice:max_slice]

            #get corresponding model list
            model_list_batch = [dict(model[min_slice:max_slice]) for model in model_list]            


            #GET NUMBER CORRECT ASSIGNMENTS IN BATCH
            hard, soft = voting_classifier_dense_preds(pred_dense_model_list=model_list_batch, ids_array=ids_batch,
                                 weights_list=weights_list, max_samples=None)
            batch_correct = return_model_stats(soft, ids_array=ids_batch, max_samples=None)[1]
            
            """PERFORM STOCHASTIC GRADIENT DESCENT; get new model weights"""
            ### If moving up the model_weight of an individual model results in a greater batch_correct...
            ##### then increase the model_weight (decrementing others)
            
            #get initial weights assigned to model ensemble
            counter = 0
            #for each model, perform stochastic gradient descent
            for model_preds in model_list_batch :
                #Modifies from weights style [0.2, 0.2, 0.2, 0.2, 0.2]
                #to [0.201, 0.1998, 0.1998, 0.1998, 0.1998] ... then bumps up weight by epsilon (if improves)
                
                #reset starting point to weights_list. #MUST MAKE COPY OF LIST; cannot point to original
                stochastic_weights_list = list(weights_list)
                
                #get reduction amount to apply to non-target weights
                above_model_lower_limit = sum(w>=model_lower_limit for w in stochastic_weights_list)
                if stochastic_weights_list[counter] >= model_lower_limit :
                    above_model_lower_limit -= 1
                    #exclude target weight from count

                if above_model_lower_limit == 0 :
                    #don't want to do any modifications if none to decrement... should be rate
                    continue
                reduction_amount_step = weight_step/float(above_model_lower_limit)
                reduction_amount_epsilon = epsilon/float(above_model_lower_limit)
                
                #increment stochastic_weights_list by WEIGHTS_STEP
                if stochastic_weights_list[counter] <= model_upper_limit :
                    stochastic_weights_list[counter] += weight_step
                else :
                    #move to next model if beyond upper limit
                    continue
                    
                #decrement non-target weights according to WEIGHTS_STEP
                for w_index, weight in enumerate(stochastic_weights_list) :
                    if w_index == counter :
                        pass
                    else :
                        if stochastic_weights_list[w_index] >= model_lower_limit :
                            stochastic_weights_list[w_index] = weight-reduction_amount_step

                #normalize back to 1 (can sometimes end up 1.00000000001 due to rounding)
                stochastic_weights_list = np.asarray(stochastic_weights_list)
                stochastic_weights_list /= sum(stochastic_weights_list)
                stochastic_weights_list.tolist()                
                
                #run through voting_classifier
                hard, soft = voting_classifier_dense_preds(pred_dense_model_list=model_list_batch, ids_array=ids_batch,
                                     weights_list=stochastic_weights_list, max_samples=None)
                stochastic_batch_correct = return_model_stats(soft, ids_array=ids_batch, max_samples=None)[1]
                                                
                #update weight_list by EPSILON when stochastic gets better score than original batch score
                if stochastic_batch_correct > batch_correct :
                    weights_list[counter] += epsilon

                    #decrement non-target weights
                    for w_index, weight in enumerate(weights_list) :
                        if w_index == counter :
                            pass
                        else :
                            if weights_list[w_index] >= model_lower_limit :
                                weights_list[w_index] = weight-reduction_amount_epsilon
                    
                    #normalize weights_list:
                    weights_list=np.asarray(weights_list)
                    weights_list /= weights_list.sum()
                    weights_list.tolist()
                                                                
                counter += 1

            #increment/move to next batch slice
            min_slice += batch_size
            max_slice += batch_size
        
        iteration_counter += 1
        print "ITERATION #:", iteration_counter
        print "WEIGHTS:", weights_list
    
    #normalize back to 1 (can sometimes end up 1.00000000001 due to rounding)
    weights_list = np.asarray(weights_list)
    weights_list /= sum(weights_list)
    weights_list.tolist()
    return weights_list


### RETURN OPTIMAL WEIGHTS (PER GRADIENT DESCENT)
 - Note: Only use to find optimal models in test set; not in Eval set. Must reuse optimal weights from test set.

MODEL NUMBERS: (0, 3, 4)
MODEL NUMBERS: (0, 2, 3, 4)
MODEL NUMBERS: (3, 4)
MODEL NUMBERS: (0, 2, 3, 4, 5)
MODEL NUMBERS: (0, 2, 3, 5)
all models - [0.02477381, 0.03094048,0.02444048,0.28677381,0.22635714,0.23502381,0.17169048]
[0,2,3,4] = [0.205,0.23166667,0.225,0.33833333]

In [469]:
# model_nums=[1, 2, 3, 4, 5]
model_nums=[0, 2, 3, 4]# - include all

#BASELINE
# optimal_weights = get_optimal_weights_soft(model_nums=model_nums, model_map=model_map, model_name_map=model_name_map, 
#                          correct_labels=var_test_ids, 
#                         weight_step=0.025, epsilon=0.01, num_samples=None, num_epochs=10, batch_size=10,
#                                           random_init=False, random_sampling=False)



#USE
optimal_weights = get_optimal_weights_soft(model_nums=model_nums, model_map=model_map, model_name_map=model_name_map, 
                         correct_labels=var_test_ids, 
                        weight_step=0.025, epsilon=0.005, num_samples=None, num_epochs=3, batch_size=10,
                                          random_init=False, random_sampling=False)



ITERATION #: 1
WEIGHTS: [0.25, 0.25, 0.25, 0.25]
ITERATION #: 2
WEIGHTS: [0.25, 0.25, 0.25, 0.25]
ITERATION #: 3
WEIGHTS: [0.25, 0.25, 0.25, 0.25]


### RETURN KEY MODEL STATS WITH OPTIMAL WEIGHTS INPUT

In [461]:
def return_models_from_nums(model_nums, model_map) :
    #get models, make sure IDs are in sorted order
    model_list = [model_map[num] for num in model_nums]
    return model_list

def return_model_names_from_nums(model_nums, model_name_map) :
    model_names_list = [model_name_map[num] for num in model_nums]
    return model_names_list


### RUN KEY MODEL STATS WITH OPTIMAL WEIGHTS INPUT (FROM TEST) ON OPTIMAL ENSEMBLE (FROM TEST) AND ALL-MODEL ENSEMBLE

In [462]:
## Soft voting classifier - VARIABLE INPUT
model_list = return_models_from_nums(model_nums=model_nums, model_map=model_map)
model_names_list = return_model_names_from_nums(model_nums=model_nums, model_name_map=model_name_map)

hard, soft = voting_classifier_dense_preds(pred_dense_model_list=model_list, ids_array=var_test_ids,
                                     weights_list=optimal_weights, max_samples=None)
print "SOFT VOTING CLASSIFIER - VARIABLE INPUT:"
print model_names_list
print_model_stats(soft, var_test_ids, max_samples=None)

SOFT VOTING CLASSIFIER - VARIABLE INPUT:
['unigram_pred_test_dense', 'trigram_pred_test_dense', 'unigram_POS_pred_test_dense', 'bigram_POS_pred_test_dense']
NUM SAMPLES:			2532
CORRECT PREDS:			2 /    0.1%
CORRECT PRED WEIGHT ASSIGNED:	0.0 /    0.0%
TOTAL WEIGHT TO CORRECT SENSE:	0.0 /    0.0%


In [ ]:
model_nums=[0, 2, 3, 4]# - include all
optimal_weights = [0.205,0.23166667,0.225,0.33833333]

model_list = return_models_from_nums(model_nums=model_nums, model_map=model_map)
model_names_list = return_model_names_from_nums(model_nums=model_nums, model_name_map=model_name_map)

hard, soft = voting_classifier_dense_preds(pred_dense_model_list=model_list, ids_array=var_test_ids,
                                     weights_list=optimal_weights, max_samples=None)
print "SOFT VOTING CLASSIFIER STATS (OPTIMAL ENSEMBLE):"
print model_names_list
print_model_stats(soft, var_test_ids, max_samples=None)


In [ ]:
model_nums=[0, 1, 2, 3, 4, 5, 6]# - include all
optimal_weights = [0.02477381, 0.03094048,0.02444048,0.28677381,0.22635714,0.23502381,0.17169048]

model_list = return_models_from_nums(model_nums=model_nums, model_map=model_map)
model_names_list = return_model_names_from_nums(model_nums=model_nums, model_name_map=model_name_map)

hard, soft = voting_classifier_dense_preds(pred_dense_model_list=model_list, ids_array=var_test_ids,
                                     weights_list=optimal_weights, max_samples=None)
print "SOFT VOTING CLASSIFIER STATS (ALL ENSEMBLES):"
print model_names_list
print_model_stats(soft, var_test_ids, max_samples=None)


### FUNCTION TO SEE ALL PREDICTED INDEXES BY MODEL OR MODEL ENSEMBLE

In [ ]:
def return_id_preds(model_nums, model_map, weights_list=None) :
    
    num_models = len(model_nums)
    
    if weights_list is None:
        weights_list = [1/float(num_models) for m in model_map]
    
    pred_dense_model_list = [model_map[num] for num in model_nums]
    
    max_indexes = []
    
    combined_preds = defaultdict(lambda: defaultdict(lambda: 0.0))
    
    model_counter = 0
    #first, iterate through individual models, add preds by index to dict
    for model_preds in pred_dense_model_list :
        #get key info from each predictive model output, append to a default_dict to be referenced later
        
        counter = 0
        #run as soft classifier (weighted)
        for (instance_id, preds) in sorted(model_preds.iteritems()) :            
            
            if preds is None :
                #handle None from LSTM
                combined_preds[counter][0] += 0
                counter += 1
                continue
            
            else :
                for ws_index, prob in preds.iteritems() :
                    combined_preds[counter][ws_index] += prob*weights_list[model_counter]
                counter += 1
        model_counter += 1
                    
    #then, iterate through dict created and append max to max_index
    for (ws_index, preds) in sorted(combined_preds.iteritems()) :            
        if preds is None :
            #handle None from LSTM
            max_index = None
        else :
            max_index = max(sorted(preds.iteritems()), key=operator.itemgetter(1))[0]
        max_indexes.append(max_index)
    max_indexes = np.asarray(max_indexes)
    
    return max_indexes, model_nums
    
    return max_indexes, model_nums
model_list = [unigram_pred_test_dense,unigram_POS_pred_test_dense]
print len(return_id_preds(model_nums=[1,2,3], model_map=model_map, weights_list=None)[0])
print len(return_id_preds(model_nums=[6], model_map=model_map, weights_list=None)[0])
print len(return_id_preds(model_nums=[0], model_map=model_map, weights_list=None)[0])
print len(lstm_dense_preds_test_matched_index.keys())
print len(lstm_dense_preds_test_matched_index.keys())

### Compare Similarity of predictions between models

In [467]:

#get the rate of same predictions between two models / model ensembles
def compare_preds(return_ids_1, return_ids_2, model_name_map) :
    ids_1 = return_ids_1[0]
    ids_2 = return_ids_2[0]
    
    model_nums_1 = return_ids_1[1]
    model_nums_2 = return_ids_2[1]
    array_length = len(ids_1)
    same_preds = np.sum(ids_1 == ids_2)
    same_preds_rate = float(same_preds)/float(array_length)
    
    return same_preds_rate, model_nums_1, model_nums_2
  
#print output by model combination
# for combo in itertools.combinations(model_map.keys(), 2):
#     model_1 = [combo[0]]
#     model_2 = [combo[1]]
#     print compare_preds(return_id_preds(model_nums=model_1, model_map=model_map, weights_list=None), 
#               return_id_preds(model_nums=model_2, model_map=model_map, weights_list=None), 
#               model_name_map=model_name_map)

  
# #Compare Single Model example
# print "\nMFS-LSTM:", compare_preds(return_id_preds(model_nums=[0], model_map=model_map, weights_list=None), 
#               return_id_preds(model_nums=[6], model_map=model_map, weights_list=None), 
#               model_name_map=model_name_map), len(var_test_ids)
  
#ensembles
# print "\nMFS-OPTIMAL ENSEMBLE:", compare_preds(return_id_preds(model_nums=[0,2,3,4], model_map=model_map, weights_list=[0.205,0.23166667,0.225,0.33833333]), 
#               return_id_preds(model_nums=[0], model_map=model_map, weights_list=None), 
#               model_name_map=model_name_map)

print "\nMFS-OPTIMAL ENSEMBLE:", compare_preds(return_id_preds(model_nums=[0,1,2,3,4,5,6], model_map=model_map, weights_list=[0.02477381, 0.03094048,0.02444048,0.28677381,0.22635714,0.23502381,0.17169048]), 
              return_id_preds(model_nums=[0], model_map=model_map, weights_list=None), 
              model_name_map=model_name_map)

print "\nMFS-OPTIMAL ENSEMBLE:", compare_preds(return_id_preds(model_nums=[0,1,2,3,4,5,6], model_map=model_map, weights_list=[0.02477381, 0.03094048,0.02444048,0.28677381,0.22635714,0.23502381,0.17169048]), 
              return_id_preds(model_nums=[1], model_map=model_map, weights_list=None), 
              model_name_map=model_name_map)

print "\nMFS-OPTIMAL ENSEMBLE:", compare_preds(return_id_preds(model_nums=[0,1,2,3,4,5,6], model_map=model_map, weights_list=[0.02477381, 0.03094048,0.02444048,0.28677381,0.22635714,0.23502381,0.17169048]), 
              return_id_preds(model_nums=[2], model_map=model_map, weights_list=None), 
              model_name_map=model_name_map)

print "\nMFS-OPTIMAL ENSEMBLE:", compare_preds(return_id_preds(model_nums=[0,1,2,3,4,5,6], model_map=model_map, weights_list=[0.02477381, 0.03094048,0.02444048,0.28677381,0.22635714,0.23502381,0.17169048]), 
              return_id_preds(model_nums=[3], model_map=model_map, weights_list=None), 
              model_name_map=model_name_map)

print "\nMFS-OPTIMAL ENSEMBLE:", compare_preds(return_id_preds(model_nums=[0,1,2,3,4,5,6], model_map=model_map, weights_list=[0.02477381, 0.03094048,0.02444048,0.28677381,0.22635714,0.23502381,0.17169048]), 
              return_id_preds(model_nums=[4], model_map=model_map, weights_list=None), 
              model_name_map=model_name_map)

print "\nMFS-OPTIMAL ENSEMBLE:", compare_preds(return_id_preds(model_nums=[0,1,2,3,4,5,6], model_map=model_map, weights_list=[0.02477381, 0.03094048,0.02444048,0.28677381,0.22635714,0.23502381,0.17169048]), 
              return_id_preds(model_nums=[5], model_map=model_map, weights_list=None), 
              model_name_map=model_name_map)

print "\nMFS-OPTIMAL ENSEMBLE:", compare_preds(return_id_preds(model_nums=[0,1,2,3,4,5,6], model_map=model_map, weights_list=[0.02477381, 0.03094048,0.02444048,0.28677381,0.22635714,0.23502381,0.17169048]), 
              return_id_preds(model_nums=[6], model_map=model_map, weights_list=None), 
              model_name_map=model_name_map)


MFS-OPTIMAL ENSEMBLE: (0.794054054054054, [0, 1, 2, 3, 4, 5, 6], [0])

MFS-OPTIMAL ENSEMBLE: (0.7967567567567567, [0, 1, 2, 3, 4, 5, 6], [1])

MFS-OPTIMAL ENSEMBLE: (0.7902702702702703, [0, 1, 2, 3, 4, 5, 6], [2])

MFS-OPTIMAL ENSEMBLE: (0.865945945945946, [0, 1, 2, 3, 4, 5, 6], [3])

MFS-OPTIMAL ENSEMBLE: (0.885945945945946, [0, 1, 2, 3, 4, 5, 6], [4])

MFS-OPTIMAL ENSEMBLE: (0.8708108108108108, [0, 1, 2, 3, 4, 5, 6], [5])

MFS-OPTIMAL ENSEMBLE: (0.4016216216216216, [0, 1, 2, 3, 4, 5, 6], [6])


In [468]:

#get the rate of same predictions between two models / model ensembles
def compare_preds(return_ids_1, return_ids_2, model_name_map) :
    ids_1 = return_ids_1[0]
    ids_2 = return_ids_2[0]
    
    model_nums_1 = return_ids_1[1]
    model_nums_2 = return_ids_2[1]
    array_length = len(ids_1)
    same_preds = np.sum(ids_1 == ids_2)
    same_preds_rate = float(same_preds)/float(array_length)
    
    return same_preds_rate, model_nums_1, model_nums_2
  
#print output by model combination
# for combo in itertools.combinations(model_map.keys(), 2):
#     model_1 = [combo[0]]
#     model_2 = [combo[1]]
#     print compare_preds(return_id_preds(model_nums=model_1, model_map=model_map, weights_list=None), 
#               return_id_preds(model_nums=model_2, model_map=model_map, weights_list=None), 
#               model_name_map=model_name_map)

  
# #Compare Single Model example
# print "\nMFS-LSTM:", compare_preds(return_id_preds(model_nums=[0], model_map=model_map, weights_list=None), 
#               return_id_preds(model_nums=[6], model_map=model_map, weights_list=None), 
#               model_name_map=model_name_map), len(var_test_ids)
  
#ensembles
# print "\nMFS-OPTIMAL ENSEMBLE:", compare_preds(return_id_preds(model_nums=[0,2,3,4], model_map=model_map, weights_list=[0.205,0.23166667,0.225,0.33833333]), 
#               return_id_preds(model_nums=[0], model_map=model_map, weights_list=None), 
#               model_name_map=model_name_map)

print "\nMFS-OPTIMAL ENSEMBLE:", compare_preds(return_id_preds(model_nums=[0,2,3,4], model_map=model_map, weights_list=[0.205,0.23166667,0.225,0.33833333]), 
              return_id_preds(model_nums=[0], model_map=model_map, weights_list=None), 
              model_name_map=model_name_map)

print "\nMFS-OPTIMAL ENSEMBLE:", compare_preds(return_id_preds(model_nums=[0,2,3,4], model_map=model_map, weights_list=[0.205,0.23166667,0.225,0.33833333]), 
              return_id_preds(model_nums=[1], model_map=model_map, weights_list=None), 
              model_name_map=model_name_map)

print "\nMFS-OPTIMAL ENSEMBLE:", compare_preds(return_id_preds(model_nums=[0,2,3,4], model_map=model_map, weights_list=[0.205,0.23166667,0.225,0.33833333]), 
              return_id_preds(model_nums=[2], model_map=model_map, weights_list=None), 
              model_name_map=model_name_map)

print "\nMFS-OPTIMAL ENSEMBLE:", compare_preds(return_id_preds(model_nums=[0,2,3,4], model_map=model_map, weights_list=[0.205,0.23166667,0.225,0.33833333]), 
              return_id_preds(model_nums=[3], model_map=model_map, weights_list=None), 
              model_name_map=model_name_map)

print "\nMFS-OPTIMAL ENSEMBLE:", compare_preds(return_id_preds(model_nums=[0,2,3,4], model_map=model_map, weights_list=[0.205,0.23166667,0.225,0.33833333]), 
              return_id_preds(model_nums=[4], model_map=model_map, weights_list=None), 
              model_name_map=model_name_map)

print "\nMFS-OPTIMAL ENSEMBLE:", compare_preds(return_id_preds(model_nums=[0,2,3,4], model_map=model_map, weights_list=[0.205,0.23166667,0.225,0.33833333]), 
              return_id_preds(model_nums=[5], model_map=model_map, weights_list=None), 
              model_name_map=model_name_map)

print "\nMFS-OPTIMAL ENSEMBLE:", compare_preds(return_id_preds(model_nums=[0,2,3,4], model_map=model_map, weights_list=[0.205,0.23166667,0.225,0.33833333]), 
              return_id_preds(model_nums=[6], model_map=model_map, weights_list=None), 
              model_name_map=model_name_map)

print "\nMFS-OPTIMAL ENSEMBLE:", compare_preds(return_id_preds(model_nums=[0,2,3,4], model_map=model_map, weights_list=[0.205,0.23166667,0.225,0.33833333]), 
              return_id_preds(model_nums=[0,1,2,3,4,5,6], model_map=model_map, weights_list=[0.02477381, 0.03094048,0.02444048,0.28677381,0.22635714,0.23502381,0.17169048]), 
              model_name_map=model_name_map)


MFS-OPTIMAL ENSEMBLE: (0.8281081081081081, [0, 2, 3, 4], [0])

MFS-OPTIMAL ENSEMBLE: (0.8281081081081081, [0, 2, 3, 4], [1])

MFS-OPTIMAL ENSEMBLE: (0.8394594594594594, [0, 2, 3, 4], [2])

MFS-OPTIMAL ENSEMBLE: (0.907027027027027, [0, 2, 3, 4], [3])

MFS-OPTIMAL ENSEMBLE: (0.9340540540540541, [0, 2, 3, 4], [4])

MFS-OPTIMAL ENSEMBLE: (0.8313513513513513, [0, 2, 3, 4], [5])

MFS-OPTIMAL ENSEMBLE: (0.36162162162162165, [0, 2, 3, 4], [6])

MFS-OPTIMAL ENSEMBLE: (0.9091891891891892, [0, 2, 3, 4], [0, 1, 2, 3, 4, 5, 6])


In [ ]:
0.205,0.23166667,0.225,0.33833333]

# APPENDIX & PRIOR CODE FOR REFERENCE:

### CREATE FUNCTION TO CONVERT EXISTING DENSE (DICT-BASED) MATRIX TO A FULL SPARSE MATRIX BY *WORD SENSE INDEX NUMBER*

In [ ]:
### CREATE A WAY TO CONVERT A DENSE DICTIONARY-BASED INDEX INTO A CONSISTENT SPARSE MATRIX

import numpy as np

def dense_to_sparse(probas, vocab_length) :
    #convert dense representation of matrix into a sparse matrix by index number    
    ##NOTE: Requires word sense INDEXES, not word senses
    sparse_matrix = np.zeros(vocab_length)
    for index, prob in probas.iteritems():
        sparse_matrix[index] = prob
    return sparse_matrix

probas_tester_1_sparse = dense_to_sparse(probas_tester_1, len(ws_index))
print "FULL MATRIX:", probas_tester_1_sparse
print "SINGLE PROBA INSTANCE:", probas_tester_1_sparse[5442]
print "SUM OF TOTAL PROBA:", sum(probas_tester_1_sparse)

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

trace = go.Heatmap(z=[[1, 20, 30],
                      [20, 1, 60],
                      [30, 60, 1]])
data=[trace]
py.iplot(data, filename='basic-heatmap')

## KEYS APPENDIX:

### CREATE CONVERT TO CONSISTENT SPARSE LIST(ARRAYS), *IN CONSISTENT SORTED ORDER*
 - NOTE: May not be needed since using voting classifier above to make ensembles instead of sparse arrays.

In [ ]:
# print len(unigram_pred_train_dense.keys())

"""NEED TO MAKE ITERATIVE VERSION ACROSS MODELS THAT PICKS UP WHERE LEFT OFF AFTER"""

def dense_to_sparse_probas(dense_dict, vocab_length) :
    #convert dense dictionary to sparse matrix by index number
    print vocab_length
    sparse_probas = []
    for instance_id, probas in sorted(dense_dict.iteritems()):
        sparse_probas.append(dense_to_sparse(probas, vocab_length))
    return sparse_probas

### WILL NEED TO USE STOCHASTIC GRADIENT REGRESSOR AND USE partial_fit() BY FEEDING IN BATCHES
#http://scikit-learn.org/dev/auto_examples/applications/plot_out_of_core_classification.html#example-applications-plot-out-of-core-classification-py
#http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html


#NOTE: USING TRAIN MAKES COMPUTER RUN OUT OF MEMORY
# unigram_pred_train_sparse = dense_to_sparse_probas(dense_dict=unigram_pred_train_dense, vocab_length=len(ws_index))

#NOTE: USING TEST ALSO MAKES COMPUTER RUN OUT OF MEMORY
# unigram_pred_test_sparse = dense_to_sparse_probas(dense_dict=unigram_pred_test_dense, vocab_length=len(ws_index))
# bigram_pred_test_sparse = dense_to_sparse_probas(dense_dict=bigram_pred_test_dense, vocab_length=len(ws_index))
# trigram_pred_test_sparse = dense_to_sparse_probas(dense_dict=trigram_pred_test_dense, vocab_length=len(ws_index))
# unigram_POS_pred_test_sparse = dense_to_sparse_probas(dense_dict=unigram_POS_pred_test_dense, vocab_length=len(ws_index))
# bigram_POS_pred_test_sparse = dense_to_sparse_probas(dense_dict=bigram_POS_pred_test_dense, vocab_length=len(ws_index))
# trigram_POS_pred_test_sparse = dense_to_sparse_probas(dense_dict=trigram_POS_pred_test_dense, vocab_length=len(ws_index))

# print "UNIGRAM PRED:", unigram_pred_test_sparse
# print

In [ ]:
import numpy as np
from sklearn import linear_model
n_samples, n_features = 10, 5
np.random.seed(0)
y = np.random.randn(n_samples)
print y
X = np.random.randn(n_samples, n_features)
print X
clf = linear_model.SGDRegressor()
fit = clf.fit(X, y)
print fit


In [ ]:
import numpy as np
from sklearn import linear_model
n_samples, n_features = 10, 5
np.random.seed(0)
y = [1, 2, 3, 4]
print y
X = [[1,2],
     [2,3],
     [3,4],
     [4,5]
    ]
print X
clf = linear_model.SGDRegressor()
fit = clf.fit(X, y)
print fit
clf.predict([2.5, 3.5])

In [ ]:
import numpy as np
from sklearn import linear_model
n_samples, n_features = 10, 5
np.random.seed(0)
y = [1, 2, 3]
print y
X = [[[1,2],[1,2]],
     [[1,2],[1,2]],
     [[1,2],[1,2]]
    ]
print X
clf = linear_model.SGDRegressor()
fit = clf.fit(X, y)
print fit
clf.predict([2.5, 3.5])

In [ ]:
"MODIFIED IN v4"
from collections import defaultdict

def sense_counter(keys, split_by_POS="N") :
    """RETURNS:
       sense_counts: a defaultdict of counts of sense assignments per word in keys
       
       NOTE: When 2+ sense assignments exist for an instance, 
       returns just the first assigned instance"""
    
    sense_counts = defaultdict(dict)
    
    for key in keys:
        key=key.strip()
        #get ID, lemma, and WordNet key 
        try :
            ID, lemma_WN_key = key.split(" ")
            lemma, WN_key = lemma_WN_key.split("%")

            if split_by_POS == "Y" :  #concatenate with POS number
                lemma = lemma+WN_key[0]

            
        #note: many instances of double assignments
        except :
            #TAKES ONLY INITIAL INSTANCE OF WORDNET SENSE ASSIGNMENT WHEN THERE ARE 2+
            ID, lemma_WN_key, extras = key.split(" ",2)
            lemma, WN_key = lemma_WN_key.split("%")

            if split_by_POS == "Y" :  #concatenate with POS number
                lemma = lemma+WN_key[0]
            
        #add to dict new count
        if lemma in sense_counts.keys() :
            if WN_key in sense_counts[lemma].keys() :
                sense_counts[lemma][WN_key] += 1
            else :
                sense_counts[lemma][WN_key] = 1
        else :
            sense_counts[lemma][WN_key] = 1

    #return the original list of keys, reduced
    return sense_counts

In [ ]:
"EDITED IN V4"
print "EXAMPLE COUNT OUTPUT FOR 'work':"
semcor_sense_counts = sense_counter(semcor_keys,split_by_POS="N")
semeval2007_sense_counts = sense_counter(semeval2007_keys,split_by_POS="N")

print semcor_sense_counts['work']
print semcor_sense_counts['work1']
print semeval2007_sense_counts['work']
print semeval2007_sense_counts['work1']

In [ ]:
print len(semcor_keys)
print len(semcor_sense_counts)
print len(semeval2007_keys)
print len(semeval2007_sense_counts)
# print semcor_sense_counts

In [ ]:
"MODIFIED IN v4"
import operator

def get_MFSenses(sense_counts) :
    """Get a dictionary of most frequent senses per word in the data set
    
    RETURNS:
    max_sense_dict: a dict of each word and corresponding Most Frequent Sense
    """
    
    MFS_dict = {}
    for word, word_senses in sense_counts.iteritems() :
        #each word_senses comes in format {'1:04:00::': 86, '2:35:02::': 20, '2:41:03::': 32, '1:04:01::': 27, '2:41:02::': 70, '1:09:00::': 11, '2:41:04::': 4, '1:06:00::': 76, '2:36:01::': 3, '2:41:05::': 3, '2:36:00::': 9, '2:38:02::': 2, '2:41:01::': 2, '2:36:09::': 2, '2:41:00::': 62, '1:06:01::': 3, '2:38:00::': 5, '2:29:00::': 9, '1:19:00::': 5}
        try:
            #get maximum sense
            max_sense = max(word_senses.iteritems(), key=operator.itemgetter(1))[0]
            #add maximum sense into dict
            MFS_dict[word] = max_sense
        except:
            pass
    
    return MFS_dict

In [ ]:
print "EXAMPLE MFS:"

semcor_MFS = get_MFSenses(semcor_sense_counts)
semeval2007_MFS = get_MFSenses(semeval2007_sense_counts)

print semcor_MFS['work']
# print semeval2007_MFS['work']

In [ ]:
"""MODIFIED IN V3"""

#GET RANDOM ASSIGNMENT AND MOST FREQUENT COUNTS

def get_ambiguity(sense_counts):
    """Get ambiguity and random baseline figures.
    
    RETURNS:
    ambiguity: average number of senses per word
    random_baseline: % correct of eval set when randomly guessing sense (based on training data)
    """
    lemma_count = 0
    total_count = 0
    total_ambiguity = 0

    #made up of lemma:{senses} where senses have counts
    #single senses form: {'1:04:00::': 86, '2:35:02::': 20, '2:41:03::': 32, '1:04:01::': 27, '2:41:02::': 70, '1:09:00::': 11, '2:41:04::': 4, '1:06:00::': 76, '2:36:01::': 3, '2:41:05::': 3, '2:36:00::': 9, '2:38:02::': 2, '2:41:01::': 2, '2:36:09::': 2, '2:41:00::': 62, '1:06:01::': 3, '2:38:00::': 5, '2:29:00::': 9, '1:19:00::': 5}
    #effectively gets to going through the data set, totaling the number of senses per lemma encountered, adding (weighting takes care of)
    for lemma, senses in sense_counts.iteritems():
        lemma_count += 1
        lemma_ambiguity = len(senses.values())
        lemma_weighting = sum(senses.values())
        total_ambiguity += lemma_ambiguity*lemma_weighting
        total_count += lemma_weighting

    ambiguity = float(total_ambiguity) / float(total_count)
    random_baseline =  1/ambiguity

    return ambiguity, random_baseline

print "AMBIGUITY, RANDOM GUESSING WITHIN SET:"
print get_ambiguity(semcor_sense_counts)

In [ ]:
np.sum(var_test_ids==0)

In [ ]:
1-float(np.sum(var_test_ids==0))/var_test_ids.shape[0]

In [ ]:
"""MODIFIED IN V3"""

def get_baselines(MFSenses_train, sense_counts_train, sense_counts_eval):
    """Get keys baseline figures.
    
    RETURNS:
    chooseMFS_baseline: % correct of eval set when picking most common sense (based on training data)
    ceiling_senses_baseline: % of word senses for lemmas in eval set also in train set (max possible to get right)
    ceiling_lemmas_baseline: % of lemmas in eval set also in train set
    """
    total_count = 0
    MFS_correct_count = 0
    ceiling_count = 0
        
    #made up of lemma:{senses} where senses have counts
    for lemma, senses in sense_counts_eval.iteritems():
        #lemma format: 'work'
        #senses format: '{'1:04:00::': 86, '2:35:02::': 20, '2:41:03::': 32, '1:04:01::': 27, '2:41:02::': 70, '1:09:00::': 11, '2:41:04::': 4, '1:06:00::': 76, '2:36:01::': 3, '2:41:05::': 3, '2:36:00::': 9, '2:38:02::': 2, '2:41:01::': 2, '2:36:09::': 2, '2:41:00::': 62, '1:06:01::': 3, '2:38:00::': 5, '2:29:00::': 9, '1:19:00::': 5}'
        
        for key, count in senses.iteritems():
            
            #if most common key is the current key, add to correct count
            try :
                if MFSenses_train[lemma] == key :
                    MFS_correct_count += count
            except :
                pass            
            
            if key in sense_counts_train[lemma].keys() :
                ceiling_count += count
            
        total_count += sum(senses.values())        
        
    #Calculate baseline metrics
    chooseMFS_baseline = float(MFS_correct_count) / float(total_count)   
    
    ceiling_senses_baseline = float(ceiling_count) / float(total_count)    
    ceiling_lemmas_baseline = len(set(sense_counts_eval.keys())&set(MFSenses_train.keys()))/float(len(set(sense_counts_eval.keys())))
    
    return chooseMFS_baseline, ceiling_senses_baseline, ceiling_lemmas_baseline

print get_baselines(semcor_MFS,semcor_sense_counts, semeval2007_sense_counts)

In [ ]:
"""ADDED IN V4"""
###SEMCOR BASIS (TAKES A FEW MINUTES)
###GET AND PRINT BASELINE METRICS
names = ['semcor', 'semeval2007','semeval2013','semeval2015','senseval2','senseval3','semcor_train', 'semcor_test']
train_data = semcor_keys
eval_data = [semcor_keys, semeval2007_keys, semeval2013_keys, semeval2015_keys, senseval2_keys, senseval3_keys,
             semcortrain_keys,semcortest_keys] 

#get training counts, most frequent terms
sense_counts_train = sense_counter(train_data)
MFSenses_train = get_MFSenses(sense_counts_train)

#get eval metrics by comparing against train data
for e, n in zip(eval_data, names) :
    sense_counts_eval = sense_counter(e)
    MFSenses_eval = get_MFSenses(sense_counts_eval)
    MFS_baseline, ceiling_senses_baseline, ceiling_lemmas_baseline = get_baselines(
        MFSenses_train,sense_counts_train,sense_counts_eval)
    ambiguity = get_ambiguity(sense_counts_eval)
    print "FOR:", n
    print "AMBIGUITY:", ambiguity
    print "MFS BASELINE:", MFS_baseline
    print "CEILING BASELINE (SENSE BASIS):", ceiling_senses_baseline
    print "CEILING BASELINE (LEMMA BASIS):", ceiling_lemmas_baseline
    print 


In [ ]:
"""ADDED IN V5"""
###SEMCOR+OMSTI BASIS (TAKES A WHILE TO RUN - COMMENT OUT IF NOT NEEDED)
###GET AND PRINT BASELINE METRICS
names = ['semcor_omsti', 'semeval2007','semeval2013','semeval2015','senseval2','senseval3']
train_data = semcor_omsti_keys
eval_data = [semcor_omsti_keys, semeval2007_keys, semeval2013_keys, semeval2015_keys, senseval2_keys, senseval3_keys] 

#get training counts, most frequent terms
sense_counts_train = sense_counter(train_data)
MFSenses_train = get_MFSenses(sense_counts_train)

#get eval metrics by comparing against train data
for e, n in zip(eval_data, names) :
    sense_counts_eval = sense_counter(e)
    MFSenses_eval = get_MFSenses(sense_counts_eval)
    MFS_baseline, ceiling_senses_baseline, ceiling_lemmas_baseline = get_baselines(
        MFSenses_train,sense_counts_train,sense_counts_eval)
    ambiguity = get_ambiguity(sense_counts_eval)
    print "FOR:", n
    print "AMBIGUITY:", ambiguity
    print "MFS BASELINE:", MFS_baseline
    print "CEILING BASELINE (SENSE BASIS):", ceiling_senses_baseline
    print "CEILING BASELINE (LEMMA BASIS):", ceiling_lemmas_baseline
    print 


## ROOT APPENDIX:

In [ ]:
# data=('semcor_root','semeval2007_root','semeval2013_root','semeval2015_root','senseval2_root','senseval3_root')
data=(semcor_root,semcortrain_root,semcortest_root,semeval2007_root,semeval2013_root,semeval2015_root,senseval2_root,senseval3_root,)
for h in data:
    sents = 0
    docs = 0
    tokens = 0
    anno = 0
    for i in h.getchildren():
        docs += 1
        for j in i.getchildren():
            sents += 1
            tokens += len(j.getchildren())
            for k in j.getchildren():
                if k.tag == 'instance':
                    anno+=1
    print '-'*25
    print h.attrib['source']
    print '-'*25
    print "Documents: ",docs
    print "Sentences: ",sents
    print "Tokens: ",tokens
    print "Annotations: ",anno
    print '-'*25
    print '\n'


In [ ]:
root.getchildren()[0].getchildren()[0].getchildren()[0].tag

In [ ]:
"""NOTE: STILL NOT PICKING UP WORD AT THE END OF THE INSTANCE! NEED TO DO SO, RIGHT? OTHERWISE JUST USING LEMMA"""

In [ ]:
print "STRUCTURE OF XML DATA FILE"

print "TAG:", root.tag
print "ATTRIB:", root.attrib
print 
print "LEVEL 1 CHILDREN (source documents):"
for l1_child in root[0:5] :
    print "L1 TAG:", l1_child.tag,"\tL1 ATTRIB:", l1_child.attrib
    for l2_child in l1_child [0:5]:
        print "\tL2 TAG:", l2_child.tag,"\tL2 ATTRIB:", l2_child.attrib
        for l3_child in l2_child [0:5]:
            print "\t\tL3 TAG:", l3_child.tag,"\tL3 ATTRIB:", l3_child.attrib,"\tL3 Text:", l3_child.text

In [ ]:
print "EXAMPLE, LEVEL 2, GET ID:"
# print semcor_root[0][0].attrib['id']
print root[0][0].attrib['id']

In [ ]:
#print all in order (will freeze up comp if not just a few examples)
for text in root[0].iter():
    print text.tag, text.attrib

In [ ]:
#to limit just to instances with lemma 'service' - two ways:
for node in root.iter('instance'):
    lemma = node.attrib.get('lemma')
    pos = node.attrib.get('pos')
    ID = node.attrib.get('id')

    if lemma == 'service':
        print lemma, pos, ID

print       

for node in root.findall('.//instance'):
    lemma = node.attrib.get('lemma')
    pos = node.attrib.get('pos')
    ID = node.attrib.get('id')
    if lemma == 'service':
        print lemma, pos, ID

In [ ]:
"""ADDED IN v5"""

#PULLS IN THE ACTUAL TEXT WORD FOR LEMMA - CAN BE USEFUL TO BUILD DICTIONARIES
for node in root.findall('.//instance'):
    lemma = node.attrib.get('lemma')
    pos = node.attrib.get('pos')
    ID = node.attrib.get('id')
    text = node.text
    if lemma == 'work':
        print lemma, pos, ID, text

In [ ]:
#make defaultdict for each instanceid
from collections import defaultdict
instance_dict = defaultdict(dict)

for node in root.findall('.//instance'):
    lemma = node.attrib.get('lemma')
    pos = node.attrib.get('pos')
    ID = node.attrib.get('id')
    instance_dict['id'] = ID
    instance_dict[ID] = {'lemma':lemma, 'pos':pos}

In [ ]:
#get single instance from created dictionary
print instance_dict['d000.s000.t005']

In [ ]:
from nltk.corpus import wordnet as wn

In [ ]:
#if get error with synsets, uncomment/run the following lines (takes a while)
# import nltk
# nltk.download()

wn.synsets("service")